Practicumopdrachten Week 4.3 (eerste kans)

In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np
import re
import sys

# Mssql database
Mssql server database

In [2]:
# database name
DB = {
    'servername': '(local)\\SQLEXPRESS',
    'database': 'DEDS_DataWarehouse'}

## Database aanmaken

In [3]:
# Connect to the SQL Server database, not a specific database but the master
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=master;Trusted_Connection=yes')

# Create a cursor object
cur = conn.cursor()

try:
    # Kill all active connections to the database
    cur.execute("""
        DECLARE @kill varchar(8000) = '';
        SELECT @kill = @kill + 'kill ' + convert(varchar, spid) + ';'
        FROM master.dbo.sysprocesses
        WHERE dbid = db_id('{database}')
        AND spid <> @@spid;
        EXEC(@kill);
    """.format(database=DB['database']))
    conn.commit()

    # Close the current connection
    cur.close()
    conn.close()

    # Reconnect to the SQL Server database
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=master;Trusted_Connection=yes')
    cur = conn.cursor()

    # Read the SQL script file
    with open('src/DEDS_DataWarehouse_Creation_Script.sql', 'r') as file:
        sql_script = file.read()

    # Split the SQL script into individual statements
    sql_statements = re.split(r'GO\n', sql_script)

    # Execute the DROP DATABASE statement
    drop_database_stmt = sql_statements[0]
    cur.execute(drop_database_stmt)

    # Execute the CREATE DATABASE statement
    create_database_stmt = sql_statements[1]
    cur.execute(create_database_stmt)

    # Start a new transaction
    conn.autocommit = False

    # Execute the remaining SQL statements within the transaction
    for sql_statement in sql_statements[2:]:
        cur.execute(sql_statement)
    # Commit the transaction
    conn.commit()

except pyodbc.Error as e:
    print(f"Error: {e}")
    conn.rollback()
    sys.exit(1)

finally:
    # Close the cursor
    cur.close()

    # Close the connection
    conn.close()

## Database connectie

In [4]:
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')


# Create a cursor from the connection
export_cursor = export_conn.cursor()

# check if connection is successful, else throw an error
if export_conn:
    print("Connection with database is established")
    # setting the autocommit to True
    export_conn.autocommit = True
else:
    print("Connection with database is not established")
    raise Exception("Connection with database is not established")


Connection with database is established


# Brontabellen
importeer de brontabellen uit de brondata folder

In [5]:
# Connectie met sqlite databases
go_crm_connection = sqlite3.connect('data/raw/go_crm.sqlite')
go_sales_connection = sqlite3.connect('data/raw/go_sales.sqlite')
go_staff_connection = sqlite3.connect('data/raw/go_staff.sqlite')


In [6]:
# inlezen csv bestanden
go_sales_inventory_levels = pd.read_csv('data/raw/GO_SALES_INVENTORY_LEVELSData.csv')
go_sales_product_forcast = pd.read_csv('data/raw/GO_SALES_PRODUCT_FORECASTData.csv')

## Inlezen van de sqlite tabellen

### go_crm

In [7]:
go_crm_age_group = pd.read_sql_query("SELECT * FROM age_group", go_crm_connection)
go_crm_country = pd.read_sql_query("SELECT * FROM country", go_crm_connection)
go_crm_retailer = pd.read_sql_query("SELECT * FROM retailer", go_crm_connection)
go_crm_retailer_contact = pd.read_sql_query("SELECT * FROM retailer_contact", go_crm_connection)
go_crm_retailer_headquarters = pd.read_sql_query("SELECT * FROM retailer_headquarters", go_crm_connection)
go_crm_retailer_segment = pd.read_sql_query("SELECT * FROM retailer_segment", go_crm_connection)
go_crm_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", go_crm_connection)
go_crm_retailer_type = pd.read_sql_query("SELECT * FROM retailer_type", go_crm_connection)
go_crm_sales_demographic = pd.read_sql_query("SELECT * FROM sales_demographic", go_crm_connection)
go_crm_sales_territory = pd.read_sql_query("SELECT * FROM sales_territory", go_crm_connection)

### go_staff

In [8]:
go_staff_course = pd.read_sql_query("SELECT * FROM course", go_staff_connection)
go_staff_sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", go_staff_connection)
go_staff_sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", go_staff_connection)
go_staff_satisfaction = pd.read_sql_query("SELECT * FROM satisfaction", go_staff_connection)
go_staff_satisfaction_type = pd.read_sql_query("SELECT * FROM satisfaction_type", go_staff_connection)
go_staff_training = pd.read_sql_query("SELECT * FROM training", go_staff_connection)

### go_sales

In [9]:
go_sales_country = pd.read_sql_query("SELECT * FROM country", go_sales_connection)
go_sales_order_details = pd.read_sql_query("SELECT * FROM order_details", go_sales_connection)
go_sales_order_header = pd.read_sql_query("SELECT * FROM order_header", go_sales_connection)
go_sales_order_method = pd.read_sql_query("SELECT * FROM order_method", go_sales_connection)
go_sales_product = pd.read_sql_query("SELECT * FROM product", go_sales_connection)
go_sales_product_line = pd.read_sql_query("SELECT * FROM product_line", go_sales_connection)
go_sales_product_type = pd.read_sql_query("SELECT * FROM product_type", go_sales_connection)
go_sales_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", go_sales_connection)
go_sales_return_reason = pd.read_sql_query("SELECT * FROM return_reason", go_sales_connection)
go_sales_returned_item = pd.read_sql_query("SELECT * FROM returned_item", go_sales_connection)
go_sales_sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", go_sales_connection)
go_sales_sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", go_sales_connection)
go_sales_sales_target_data = pd.read_sql("SELECT * FROM SALES_TARGETData", go_sales_connection)

# Combineren van de tabellen
Combineren van de brondata om tot ons ETL te komen

In [10]:
all_dataframes = []

### Sales_demographic

In [11]:
# merging sales_demographic and age_group
sales_demographic = pd.merge(go_crm_sales_demographic, go_crm_age_group, left_on='AGE_GROUP_CODE', right_on='AGE_GROUP_CODE')

# renaming the columns
sales_demographic = sales_demographic.rename(columns={'DEMOGRAPHIC_CODE': 'SALES_DEMOGRAPHIC_code', 'AGE_GROUP_CODE': 'SALES_DEMOGRAPHIC_AGE_GROUP_CODE', 'RETAILER_CODEMR': 'SALES_DEMOGRAPHIC_RETAILER_CODE', 'SALES_PERCENT' : 'SALES_DEMOGRAPHIC_SALES_PERCENT', 'UPPER_AGE': 'SALES_DEMOGRAPHIC_UPPER_AGE', 'LOWER_AGE': 'SALES_DEMOGRAPHIC_LOWER_AGE'})

all_dataframes.append(sales_demographic)

sales_demographic.head()

,SALES_DEMOGRAPHIC_code,SALES_DEMOGRAPHIC_RETAILER_CODE,SALES_DEMOGRAPHIC_AGE_GROUP_CODE,SALES_DEMOGRAPHIC_SALES_PERCENT,TRIAL222,SALES_DEMOGRAPHIC_UPPER_AGE,SALES_DEMOGRAPHIC_LOWER_AGE,TRIAL219
0,1,1,1,52,T,20,0,T
1,10,2,4,16,T,50,41,T
2,100,17,4,11,T,50,41,T
3,1000,164,3,6,T,40,31,T
4,1001,164,4,9,T,50,41,T


### returned_item

In [12]:
# to be used in order details
returned_item_columns = ['RETURN_CODE', 'RETURN_DATE', 'RETURN_QUANTITY', 'ORDER_DETAIL_CODE', 'RETURN_REASON_CODE', 'RETURN_DESCRIPTION_EN']
returned_item = pd.merge(go_sales_returned_item, go_sales_return_reason, left_on='RETURN_REASON_CODE', right_on='RETURN_REASON_CODE')
returned_item = pd.merge(returned_item, go_sales_order_details, left_on='ORDER_DETAIL_CODE', right_on='ORDER_DETAIL_CODE')

# filtering the columns
returned_item = returned_item[returned_item_columns]

# renaming the columns
returned_item = returned_item.rename(columns={'RETURN_CODE': 'RETURNED_ITEM_code', 'RETURN_DATE': 'RETURNED_ITEM_DATE', 'RETURN_QUANTITY': 'RETURNED_ITEM_QUANTITY', 'ORDER_DETAIL_CODE': 'RETURNED_ITEM_ORDER_DETAIL_CODE', 'RETURN_REASON_CODE': 'RETURNED_ITEM_RETURN_REASON_code', 'RETURN_DESCRIPTION_EN': 'RETURNED_ITEM_RETURN_REASON_description_en'})

all_dataframes.append(returned_item)

returned_item.head()

,RETURNED_ITEM_code,RETURNED_ITEM_DATE,RETURNED_ITEM_QUANTITY,RETURNED_ITEM_ORDER_DETAIL_CODE,RETURNED_ITEM_RETURN_REASON_code,RETURNED_ITEM_RETURN_REASON_description_en
0,1491,1-8-2021 04:10:24,8,84858,5,Unsatisfactory product
1,1492,6-12-2020 18:46:19,2,84440,2,Incomplete product
2,1493,24-6-2021 09:23:14,22,84867,4,Wrong product shipped
3,1494,21-7-2021 00:00:09,20,84873,3,Wrong product ordered
4,1496,7-10-2020 05:13:58,2,84488,1,Defective product


### Unit

In [13]:
unit_columns = ['UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE', 'UNIT_id', 'ORDER_DETAIL_CODE']
unit = go_sales_order_details

# make unit_id autoincrement
unit['UNIT_id'] = range(1, len(unit) + 1)

# filtering the columns
unit = unit[unit_columns]

# renaming the columns
unit = unit.rename(columns={'UNIT_COST': 'UNIT_COST_cost', 'UNIT_PRICE': 'UNIT_PRICE_price', 'UNIT_SALE_PRICE': 'UNIT_SALE_sale'})

all_dataframes.append(unit)

unit.head()

,UNIT_COST_cost,UNIT_PRICE_price,UNIT_SALE_sale,UNIT_id,ORDER_DETAIL_CODE
0,16.1,22.54,22.54,1,100000
1,16.1,22.54,22.54,2,100001
2,16.1,22.54,22.54,3,100002
3,16.1,22.54,22.54,4,100003
4,16.1,22.54,22.54,5,100004


### Sales_staff

In [14]:
sales_staff_columns = ['SALES_STAFF_CODE', 'EMAIL', 'EXTENSION', 'POSITION_EN', 'WORK_PHONE', 'DATE_HIRED', 'MANAGER_CODE', 'FAX', 'FIRST_NAME', 'LAST_NAME',  'ADDRESS1', 'ADDRESS2', 'SALES_BRANCH_CODE']

sales_staff = pd.merge(go_staff_sales_staff, go_staff_sales_branch, left_on='SALES_BRANCH_CODE', right_on='SALES_BRANCH_CODE')

# filtering the columns
sales_staff = sales_staff[sales_staff_columns]

# renaming the columns
sales_staff = sales_staff.rename(columns={'SALES_STAFF_CODE': 'SALES_STAFF_code', 'EMAIL': 'SALES_STAFF_email', 'EXTENSION': 'SALES_STAFF_extension', 'POSITION_EN': 'SALES_STAFF_POSITION_EN_position', 'WORK_PHONE': 'SALES_STAFF_WORK_PHONE_work_phone', 'DATE_HIRED': 'SALES_STAFF_DATE_HIRED_hired', 'MANAGER_CODE': 'SALES_STAFF_MANAGER_CODE_manager', 'FAX': 'SALES_STAFF_FAX', 'FIRST_NAME': 'SALES_STAFF_FIRST_NAME_first_name', 'LAST_NAME': 'SALES_STAFF_LAST_NAME_last_name', 'ADDRESS1': 'SALES_STAFF_SALES_BRANCH_ADDRESS1_address', 'ADDRESS2': 'SALES_STAFF_SALES_BRANCH_ADDRESS2_address', 'SALES_BRANCH_CODE' : 'SALES_STAFF_SALES_BRANCH_CODE_branch_code'})

all_dataframes.append(sales_staff)

sales_staff.head()

,SALES_STAFF_code,SALES_STAFF_email,SALES_STAFF_extension,SALES_STAFF_POSITION_EN_position,SALES_STAFF_WORK_PHONE_work_phone,SALES_STAFF_DATE_HIRED_hired,SALES_STAFF_MANAGER_CODE_manager,SALES_STAFF_FAX,SALES_STAFF_FIRST_NAME_first_name,SALES_STAFF_LAST_NAME_last_name,SALES_STAFF_SALES_BRANCH_ADDRESS1_address,SALES_STAFF_SALES_BRANCH_ADDRESS2_address,SALES_STAFF_SALES_BRANCH_CODE_branch_code
0,100,TSavolainen@grtd123.com,825,Level 2 Sales Representative,+358(0)17 - 433 127,23-Jul-1998 12:00:00 AM,18,+358(0)17 - 433 129,Tuomas,Savolainen,Kauppakatu 33,None,31
1,101,CKim@grtd123.com,1228,Level 2 Sales Representative,82-2-778-6587,15-Dec-1998 12:00:00 AM,87,82-2-778-6586,Chang-ho,Kim,"234-12, Kongdeok-Dong",Mapo-Ku,32
2,102,JChoi@grtd123.com,2641,Level 3 Sales Representative,82-2-778-6587,27-Nov-1998 12:00:00 AM,87,82-2-778-6586,Jung-ho,Choi,"234-12, Kongdeok-Dong",Mapo-Ku,32
3,103,KKwao@grtd123.com,1541,Level 2 Sales Representative,886-2-2975-1988,07-Oct-1998 12:00:00 AM,87,886-2-2989-4978,Kwei-Ping,Kao,"154, Choung-Cheng South Road",None,33
4,104,CFon@grtd123.com,2311,Level 3 Sales Representative,886-2-2975-1988,08-May-1996 12:00:00 AM,87,886-2-2989-4978,Chin-Tsai,Fang,"154, Choung-Cheng South Road",None,33


### Satisfaction_type

In [15]:
satisfaction_type = go_staff_satisfaction_type

# rename the columns
satisfaction_type = satisfaction_type.rename(columns={'SATISFACTION_TYPE_CODE': 'SATISFACTION_TYPE_code', 'DESCRIPTION': 'SATISFACTION_TYPE_description'})

all_dataframes.append(satisfaction_type)

satisfaction_type.head()

,SATISFACTION_TYPE_code,SATISFACTION_TYPE_DESCRIPTION,TRIAL633
0,1,Not satisfied,T
1,2,Less than satisfied,T
2,3,Satisfied,T
3,4,Very Satisfied,T
4,5,More than satisfied,T


### Course

In [16]:
course = go_staff_course

# rename the columns
course = course.rename(columns={'COURSE_CODE': 'COURSE_code', 'DESCRIPTION': 'COURSE_description'})

all_dataframes.append(course)

course.head()

,COURSE_code,COURSE_DESCRIPTION,TRIAL633
0,1,GO Orientation,T
1,2,GO Communication,T
2,3,GO Sales 1,T
3,4,GO Sales 2,T
4,5,GO Marketing 1,T


### Year

In [17]:
# to be used in training and satisfaction
# making year dataframe with year as the primary key
year = pd.DataFrame({'YEAR': pd.date_range(start='1/1/1900', end='1/1/2200').year})

# Drop duplicates based on 'YEAR' column
year = year.drop_duplicates()

# Set 'YEAR' as the index again
year = year.set_index('YEAR')

all_dataframes.append(year)

year.head()

""
YEAR
1900
1901
1902
1903
1904


### Date

In [18]:
# to be used in order header
date = pd.DataFrame({'DATE_date': pd.date_range(start='1/1/1900', end='1/1/2200')})

# drop duplicates
date = date.drop_duplicates()

date = date.set_index('DATE_date')

all_dataframes.append(date)

date.head()

""
DATE_date
1900-01-01
1900-01-02
1900-01-03
1900-01-04
1900-01-05


### Order

In [19]:
# to be used in order header
order_columns = ['ORDER_METHOD_CODE', 'ORDER_NUMBER', 'ORDER_METHOD_EN']
order = pd.merge(go_sales_order_header, go_sales_order_method, left_on='ORDER_METHOD_CODE', right_on='ORDER_METHOD_CODE')

#filtering the columns
order = order[order_columns]

# renaming the columns
order = order.rename(columns={'ORDER_METHOD_CODE': 'ORDER_ORDER_METHOD_CODE_method_code', 'ORDER_NUMBER': 'ORDER_order_number', 'ORDER_METHOD_EN': 'ORDER_ORDER_METHOD_EN_method'})


all_dataframes.append(order)

order.head()

,ORDER_ORDER_METHOD_CODE_method_code,ORDER_order_number,ORDER_ORDER_METHOD_EN_method
0,7,1153,Sales visit
1,4,1154,E-mail
2,7,1155,Sales visit
3,5,1156,Web
4,2,1157,Telephone


### Retailer_site

In [20]:
retailer_site = go_sales_retailer_site

# rename the columns
retailer_site = retailer_site.rename(columns={'RETAILER_SITE_CODE': 'RETAILER_SITE_code', 'COUNTRY_CODE': 'RETAILER_SITE_COUNTRY_CODE_country', 'CITY': 'RETAILER_SITE_CITY_city', 'REGION': 'RETAILER_SITE_REGION_region', 'POSTAL_ZONE': 'RETAILER_SITE_POSTAL_ZONE_postal_zone', 'RETAILER_CODE': 'RETAILER_SITE_RETAILER_CODE_retailer_code', 'ACTIVE_INDICATOR': 'RETAILER_SITE_ACTIVE_INDICATOR_indicator', 'ADDRESS1': 'RETAILER_SITE_ADDRESS1_address', 'ADDRESS2': 'RETAILER_SITE_ADDRESS2_address'})

all_dataframes.append(retailer_site)

retailer_site.head()

,RETAILER_SITE_code,RETAILER_SITE_RETAILER_CODE_retailer_code,RETAILER_SITE_ADDRESS1_address,RETAILER_SITE_ADDRESS2_address,RETAILER_SITE_CITY_city,RETAILER_SITE_REGION_region,RETAILER_SITE_POSTAL_ZONE_postal_zone,RETAILER_SITE_COUNTRY_CODE_country,RETAILER_SITE_ACTIVE_INDICATOR_indicator,TRIAL888
0,1,89,1117 Franklin Blvd,None,Winnipeg,Manitoba,R2C 0M5,4,1,T
1,10,92,"Suite 1500, 802 Docklands Street",None,Vancouver,British Columbia,V6E 3Z3,4,1,T
2,100,95,4830 Princeton,Unit 2,Albuquerque,New Mexico,87186,3,1,T
3,101,95,6630 Burnet Road,None,Austin,Texas,78769,3,1,T
4,102,108,2189 Louisville Road,None,Independence,Kentucky,41051,3,0,T


### Sales_branch

In [21]:
sales_branch = go_sales_sales_branch

# rename the columns
sales_branch = sales_branch.rename(columns={'SALES_BRANCH_CODE': 'SALES_BRANCH_code', 'COUNTRY_CODE': 'SALES_BRANCH_COUNTRY_CODE_country', 'REGION': 'SALES_BRANCH_REGION_region', 'CITY': 'SALES_BRANCH_CITY_city', 'POSTAL_ZONE': 'SALES_BRANCH_POSTAL_ZONE_postal_zone', 'ADDRESS1': 'SALES_BRANCH_ADDRESS1_address', 'ADDRESS2': 'SALES_BRANCH_ADDRESS2_address', 'MAIN_ADDRESS': 'SALES_BRANCH_MAIN_ADDRESS_address'})

all_dataframes.append(sales_branch)

sales_branch.head()

,SALES_BRANCH_code,SALES_BRANCH_ADDRESS1_address,SALES_BRANCH_ADDRESS2_address,SALES_BRANCH_CITY_city,SALES_BRANCH_REGION_region,SALES_BRANCH_POSTAL_ZONE_postal_zone,SALES_BRANCH_COUNTRY_CODE_country,TRIAL888
0,13,Schwabentor 35,None,Hamburg,None,D-22529,2,T
1,14,Leopoldstraße 36,None,München,None,D-81241,2,T
2,15,Isafjordsgatan 30 C,None,Kista,None,S-164 40,10,T
3,17,"7800, 756 - 6th Avenue. S.W.",None,Calgary,Alberta,T2P 3Z0,4,T
4,18,789 Yonge Street,None,Toronto,Ontario,M2M 4K8,4,T


### Retailer_contact

In [22]:
retailer_contact = go_crm_retailer_contact

# rename the columns
retailer_contact = retailer_contact.rename(columns={'RETAILER_CONTACT_CODE': 'RETAILER_CONTACT_code', 'E_MAIL': 'RETAILER_CONTACT_email', 'RETAILER_SITE_CODE': 'RETAILER_CONTACT_RETAILER_SITE_CODE_site_code', 'JOB_POSITION_EN': 'RETAILER_CONTACT_JOB_POSITION_EN_position', 'EXTENSION': 'RETAILER_CONTACT_EXTENSION_extension', 'FAX': 'RETAILER_CONTACT_FAX_fax', 'GENDER' : 'RETAILER_CONTACT_GENDER_gender', 'FIRST_NAME': 'RETAILER_CONTACT_FIRST_NAME_first_name', 'LAST_NAME': 'RETAILER_CONTACT_LAST_NAME_last_name'})

all_dataframes.append(retailer_contact)

retailer_contact.head()

,RETAILER_CONTACT_code,RETAILER_CONTACT_RETAILER_SITE_CODE_site_code,RETAILER_CONTACT_FIRST_NAME_first_name,RETAILER_CONTACT_LAST_NAME_last_name,RETAILER_CONTACT_JOB_POSITION_EN_position,RETAILER_CONTACT_EXTENSION_extension,RETAILER_CONTACT_FAX_fax,RETAILER_CONTACT_email,RETAILER_CONTACT_GENDER_gender,TRIAL222
0,10,14,Frank,Smith,Chief Purchaser,2489,1 (514) 741-1610,FSmith@legolfeurinc.com,M,T
1,100,85,Amanda,Yates,Stock Manager,None,1 (702) 369-2274,Y2883@emertxe.com,F,T
2,101,84,Paula,Schmidt,Assistant Purchaser,None,1 (702) 882-0978,S8832@emertxe.com,F,T
3,102,88,Micheal,Tao,District Manager,445,1 (713) 524-3215,T8839@emertxe.com,M,T
4,103,92,Jack,Murphy,District Manager,None,1 (305) 557-5478,Murphy@eyedimensions3.com,M,T


### Retailer

### Training

In [23]:
retailer = pd.merge(go_crm_retailer, go_crm_retailer_type, left_on='RETAILER_TYPE_CODE', right_on='RETAILER_TYPE_CODE')

# rename the columns
retailer = retailer.rename(columns={'RETAILER_CODE': 'RETAILER_code', 'COMPANY_NAME': 'RETAILER_name', 'RETAILER_CODEMR': 'RETAILER_COMPANY_CODE_MR_company', 'RETAILER_TYPE_CODE': 'RETAILER_RETAILER_TYPE_code', 'RETAILER_TYPE_EN': 'RETAILER_RETAILER_TYPE_EN'})

all_dataframes.append(retailer)

retailer.head()

,RETAILER_code,RETAILER_COMPANY_CODE_MR_company,RETAILER_name,RETAILER_RETAILER_TYPE_code,TRIAL219,RETAILER_RETAILER_TYPE_EN,TRIAL222
0,1,None,ActiForme,5,T,Equipment Rental Store,T
1,100,70,Golf Masters,1,T,Golf Shop,T
2,101,71,The Marketplace,2,T,Department Store,T
3,102,72,Camping Equipment Online,3,T,Direct Marketing,T
4,103,73,Tamarack Outfitter Rentals,5,T,Equipment Rental Store,T


### Product

In [24]:
product = pd.merge(go_sales_product, go_sales_product_type, left_on='PRODUCT_TYPE_CODE', right_on='PRODUCT_TYPE_CODE')
product = pd.merge(product, go_sales_product_line, left_on='PRODUCT_LINE_CODE', right_on='PRODUCT_LINE_CODE')

# rename the columns
product = product.rename(columns={'PRODUCT_NUMBER': 'PRODUCT_number', 'PRODUCT_NAME': 'PRODUCT_name_product', 'DESCRIPTION': 'PRODUCT_description_description', 'PRODUCT_IMAGE': 'PRODUCT_image_image', 'INTRODUCTION_DATE': 'PRODUCT_INTRODUCTION_DATE_introduced', 'PRODUCTION_COST': 'PRODUCT_PRODUCTION_COST_cost', 'MARGIN': 'PRODUCT_MARGIN_margin', 'LANGUAGE': 'PRODUCT_LANGUAGE_language',  'PRODUCT_TYPE_CODE': 'PRODUCT_PRODUCT_TYPE_code', 'PRODUCT_TYPE_EN': 'PRODUCT_PRODUCT_TYPE_code_en', 'PRODUCT_LINE_CODE': 'PRODUCT_PRODUCT_LINE_code', 'PRODUCT_LINE_EN': 'PRODUCT_PRODUCT_LINE_code_en'})

all_dataframes.append(product)

product.head()

,PRODUCT_number,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_image_image,PRODUCT_LANGUAGE_language,PRODUCT_name_product,PRODUCT_description_description,TRIAL888_x,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code_en,TRIAL888_y,PRODUCT_PRODUCT_LINE_code_en,TRIAL888
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,1,Cooking Gear,T,Camping Equipment,T
1,10,15-2-2011,1,10,.4,P10CE1CG1.jpg,EN,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",T,1,Cooking Gear,T,Camping Equipment,T
2,100,15-2-2011,17,3,.5,P100OP4FA17.jpg,EN,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,T,4,First Aid,T,Outdoor Protection,T
3,101,15-12-2019,18,305.54,.43,P101GE5IR18.jpg,EN,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,T,5,Irons,T,Golf Equipment,T
4,102,10-12-2019,18,380.95,.51,P102GE5IR18.jpg,EN,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,T,5,Irons,T,Golf Equipment,T


### Order_details

In [25]:
order_details = go_sales_order_details

# merging unit and order_details
order_details = pd.merge(order_details, unit, left_on=['ORDER_DETAIL_CODE', 'UNIT_id'], right_on=['ORDER_DETAIL_CODE', 'UNIT_id'])

# dropping the columns replaced by unit and returned_item
order_details = order_details.drop(columns=['UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE']) 

# rename the columns
order_details = order_details.rename(columns={'QUANTITY': 'ORDER_DETAILS_QUANTITY_quantity', 'ORDER_NUMBER': 'ORDER_DETAILS_ORDER_NUMBER_order', 'PRODUCT_NUMBER': 'ORDER_DETAILS_PRODUCT_NUMBER_product', 'UNIT_ID': 'ORDER_DETAILS_UNIT_ID_unit', 'ORDER_DETAIL_CODE' : 'ORDER_DETAIL_code'})

all_dataframes.append(order_details)

order_details.head()

,ORDER_DETAIL_code,ORDER_DETAILS_ORDER_NUMBER_order,ORDER_DETAILS_PRODUCT_NUMBER_product,ORDER_DETAILS_QUANTITY_quantity,TRIAL879,UNIT_id,UNIT_COST_cost,UNIT_PRICE_price,UNIT_SALE_sale
0,100000,8462,96,16,T,1,16.1,22.54,22.54
1,100001,9111,96,20,T,2,16.1,22.54,22.54
2,100002,8451,96,24,T,3,16.1,22.54,22.54
3,100003,8453,96,18,T,4,16.1,22.54,22.54
4,100004,8439,96,20,T,5,16.1,22.54,22.54


## training

In [26]:
training = go_staff_training

# converting the year to int 32
training['YEAR'] = training['YEAR'].astype('int32')

# replacing the original columns with year and course
# merging the year and course
training = pd.merge(training, year, left_on='YEAR', right_on='YEAR')
training = pd.merge(training, course, left_on='COURSE_CODE', right_on='COURSE_code')

# rename the columns
training = training.rename(columns={'SALES_STAFF_CODE': 'TRAINING_SALES_STAFF_CODE', 'COURSE_CODE': 'TRAINING_COURSE_CODE', 'YEAR': 'TRAINING_YEAR'})

all_dataframes.append(training)

training.head()

,TRAINING_YEAR,TRAINING_SALES_STAFF_CODE,TRAINING_COURSE_CODE,TRIAL633_x,COURSE_code,COURSE_DESCRIPTION,TRIAL633_y
0,2004,100,2,T,2,GO Communication,T
1,2004,100,3,T,3,GO Sales 1,T
2,2004,100,4,T,4,GO Sales 2,T
3,2005,100,5,T,5,GO Marketing 1,T
4,2006,100,6,T,6,GO Marketing 2,T


### Satisfaction

In [27]:
satisfaction = go_staff_satisfaction

# converting the year to int 32
satisfaction['YEAR'] = satisfaction['YEAR'].astype('int32')

# replacing the original columns with year and satisfaction_type
# merging the year and satisfaction_type
satisfaction = pd.merge(satisfaction, year, left_on='YEAR', right_on='YEAR')
satisfaction = pd.merge(satisfaction, satisfaction_type, left_on='SATISFACTION_TYPE_CODE', right_on='SATISFACTION_TYPE_code')

# rename the columns
satisfaction = satisfaction.rename(columns={'SALES_STAFF_CODE': 'SATISFACTION_SALES_STAFF_CODE', 'SATISFACTION_TYPE_CODE': 'SATISFACTION_SATISFACTION_TYPE_CODE', 'YEAR': 'SATISFACTION_YEAR'})

all_dataframes.append(satisfaction)

satisfaction.head()

,SATISFACTION_YEAR,SATISFACTION_SALES_STAFF_CODE,SATISFACTION_SATISFACTION_TYPE_CODE,TRIAL633_x,SATISFACTION_TYPE_code,SATISFACTION_TYPE_DESCRIPTION,TRIAL633_y
0,2004,100,1,T,1,Not satisfied,T
1,2004,101,4,T,4,Very Satisfied,T
2,2004,102,4,T,4,Very Satisfied,T
3,2004,103,3,T,3,Satisfied,T
4,2004,104,4,T,4,Very Satisfied,T


### Order_header

In [28]:
order_header = pd.merge(go_sales_order_header, retailer, left_on='RETAILER_NAME', right_on='RETAILER_name')

# converting the date to datetime
order_header['ORDER_DATE'] = pd.to_datetime(order_header['ORDER_DATE'], format='mixed')

# replacing the original columns with date, sales_staff, retailer_site and retailer_contact
# merging the date, sales_staff, retailer_site and retailer_contact
order_header = pd.merge(order_header, date, left_on='ORDER_DATE', right_on='DATE_date')
order_header = pd.merge(order_header, sales_staff, left_on='SALES_STAFF_CODE', right_on='SALES_STAFF_code')
order_header = pd.merge(order_header, retailer_site, left_on='RETAILER_SITE_CODE', right_on='RETAILER_SITE_code')
order_header = pd.merge(order_header, retailer_contact, left_on='RETAILER_CONTACT_CODE', right_on='RETAILER_CONTACT_code')

# rename the columns
order_header = order_header.rename(columns={'ORDER_NUMBER': 'ORDER_HEADER_number', 'SALES_STAFF_CODE': 'ORDER_HEADER_SALES_STAFF_CODE', 'SALES_BRANCH_CODE': 'ORDER_HEADER_SALES_BRANCH_CODE', 'ORDER_DATE': 'ORDER_HEADER_ORDER_DATE', 'RETAILER_SITE_CODE': 'ORDER_HEADER_RETAILER_SITE_CODE', 'RETAILER_CONTACT_CODE': 'ORDER_HEADER_RETAILER_CONTACT_CODE', 'RETAILER_CODE': 'ORDER_HEADER_RETAILER_CODE'})

all_dataframes.append(order_header)

order_header.head()

,ORDER_HEADER_number,RETAILER_NAME,ORDER_HEADER_RETAILER_SITE_CODE,ORDER_HEADER_RETAILER_CONTACT_CODE,ORDER_HEADER_SALES_STAFF_CODE,ORDER_HEADER_SALES_BRANCH_CODE,ORDER_HEADER_ORDER_DATE,ORDER_METHOD_CODE,TRIAL885,RETAILER_code,...,RETAILER_CONTACT_code,RETAILER_CONTACT_RETAILER_SITE_CODE_site_code,RETAILER_CONTACT_FIRST_NAME_first_name,RETAILER_CONTACT_LAST_NAME_last_name,RETAILER_CONTACT_JOB_POSITION_EN_position,RETAILER_CONTACT_EXTENSION_extension,RETAILER_CONTACT_FAX_fax,RETAILER_CONTACT_email,RETAILER_CONTACT_GENDER_gender,TRIAL222_y
0,1153,Ultra Sports,4,6,50,18,2020-04-14,7,T,90,...,6,4,Marie,Turcotte,Assistant Purchaser,635,1 (514) 747-2582,MTurcotte@cybserv19.com,F,T
1,1154,Ultra Sports,4,6,50,18,2020-10-01,4,T,90,...,6,4,Marie,Turcotte,Assistant Purchaser,635,1 (514) 747-2582,MTurcotte@cybserv19.com,F,T
2,1155,Ultra Sports,5,65,49,18,2020-04-21,7,T,90,...,65,5,Andrea,Martens,Stock Manager,None,1 (705) 675-9848,AMartens@cybserv19.com,F,T
3,1156,Ultra Sports,5,65,49,18,2020-09-09,5,T,90,...,65,5,Andrea,Martens,Stock Manager,None,1 (705) 675-9848,AMartens@cybserv19.com,F,T
4,1157,Ultra Sports,6,66,50,18,2020-04-10,2,T,90,...,66,6,Bill,Follett,Site Assistant Manager,None,1 (819) 427-4397,BFollett@cybserv19.com,M,T


### GO_SALES_INVENTORY_LEVELS

In [29]:
# adding a leading 0 to the month if it is a single digit
go_sales_inventory_levels['INVENTORY_MONTH'] = go_sales_inventory_levels['INVENTORY_MONTH'].astype(str).str.zfill(2)

# combine year and month to one column
go_sales_inventory_levels['YEAR_MONTH'] = go_sales_inventory_levels['INVENTORY_YEAR'].astype(str) + '-' + go_sales_inventory_levels['INVENTORY_MONTH'].astype(str)

# dropping the original columns
go_sales_inventory_levels = go_sales_inventory_levels.drop(columns=['INVENTORY_YEAR', 'INVENTORY_MONTH'])

go_sales_inventory_levels = go_sales_inventory_levels.rename(columns={'INVENTORY_COUNT': 'GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT', 'PRODUCT_NUMBER': 'GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER', 'YEAR_MONTH': 'GO_SALES_INVENTORY_LEVELS_YEAR_MONTH'})

all_dataframes.append(go_sales_inventory_levels)

go_sales_inventory_levels.head()

,GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER,GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT,GO_SALES_INVENTORY_LEVELS_YEAR_MONTH
0,48,1932,2021-04
1,49,1400,2021-04
2,50,21705,2021-04
3,51,9710,2021-04
4,52,5616,2021-04


### GO_SALES_PRODUCT_FORECAST

In [30]:
# merging the product and go_sales_product_forecast
product['PRODUCT_number'] = product['PRODUCT_number'].astype('int')
sales_product_forcast = pd.merge(go_sales_product_forcast, product, left_on='PRODUCT_NUMBER', right_on='PRODUCT_number')

# adding a leading zero to the month if it is a single digit
sales_product_forcast['MONTH'] = sales_product_forcast['MONTH'].astype(str).str.zfill(2)

#combine year and month to one column
sales_product_forcast['YEAR_MONTH'] = sales_product_forcast['YEAR'].astype(str) + '-' + sales_product_forcast['MONTH'].astype(str)

# dropping the original columns
sales_product_forcast = sales_product_forcast.drop(columns=['YEAR', 'MONTH'])

sales_product_forecast = sales_product_forcast.rename(columns={'EXPECTED_VOLUME': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME', 'EXPECTED_COST': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_COST', 'EXPECTED_MARGIN': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN', 'PRODUCT_NUMBER': 'GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER', 'YEAR_MONTH': 'GO_SALES_PRODUCT_FORECAST_YEAR_MONTH'})

all_dataframes.append(sales_product_forecast)

sales_product_forecast.head()

,GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER,GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME,PRODUCT_number,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_image_image,PRODUCT_LANGUAGE_language,PRODUCT_name_product,PRODUCT_description_description,TRIAL888_x,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code_en,TRIAL888_y,PRODUCT_PRODUCT_LINE_code_en,TRIAL888,GO_SALES_PRODUCT_FORECAST_YEAR_MONTH
0,44,383,44,10-1-2021,6,231.11,.33,P44ME2RP6.jpg,EN,Husky Rope 100,11 mm diameter standard rope. Length: 100 m. ...,T,2,Rope,T,Mountaineering Equipment,T,2022-12
1,45,80,45,10-1-2021,6,383.32,.33,P44ME2RP6.jpg,EN,Husky Rope 200,11 mm diameter standard rope. Length: 200 m. ...,T,2,Rope,T,Mountaineering Equipment,T,2021-01
2,45,51,45,10-1-2021,6,383.32,.33,P44ME2RP6.jpg,EN,Husky Rope 200,11 mm diameter standard rope. Length: 200 m. ...,T,2,Rope,T,Mountaineering Equipment,T,2021-02
3,45,214,45,10-1-2021,6,383.32,.33,P44ME2RP6.jpg,EN,Husky Rope 200,11 mm diameter standard rope. Length: 200 m. ...,T,2,Rope,T,Mountaineering Equipment,T,2021-03
4,45,300,45,10-1-2021,6,383.32,.33,P44ME2RP6.jpg,EN,Husky Rope 200,11 mm diameter standard rope. Length: 200 m. ...,T,2,Rope,T,Mountaineering Equipment,T,2021-04


###  SALES_TARGETData

In [31]:
# merge product, retailer and sales_staff
product['PRODUCT_number'] = product['PRODUCT_number'].astype('int')

go_sales_sales_target_data['PRODUCT_NUMBER'] = go_sales_sales_target_data['PRODUCT_NUMBER'].astype('int')

sales_sales_target_data = pd.merge(go_sales_sales_target_data, product, left_on='PRODUCT_NUMBER', right_on='PRODUCT_number', suffixes=('_product', '_retailer'))

sales_sales_target_data = pd.merge(sales_sales_target_data, retailer, left_on='RETAILER_CODE', right_on='RETAILER_code', suffixes=('_retailer', '_sales_staff'))

# delete the original columns that are replaced by product, retailer and sales_staff
sales_sales_target_data = sales_sales_target_data.drop(columns=['PRODUCT_NUMBER', 'RETAILER_CODE', 'RETAILER_name', 'RETAILER_COMPANY_CODE_MR_company', 'RETAILER_RETAILER_TYPE_code', 'RETAILER_RETAILER_TYPE_EN'])

sales_target_data = sales_sales_target_data.rename(columns={'SALES_YEAR': 'SALES_TARGETDATA_SALES_YEAR', 'SALES_PERIOD': 'SALES_TARGETDATA_SALES_PERIOD', 'RETAILER_NAME': 'SALES_TARGETDATA_RETAILER_NAME', 'SALES_TARGET': 'SALES_TARGETDATA_SALES_TARGET', 'TARGET_COST': 'SALES_TARGETDATA_TARGET_COST', 'TARGET_MARGIN': 'SALES_TARGETDATA_TARGET_MARGIN', 'SALES_STAFF_CODE': 'SALES_TARGETDATA_SALES_STAFF_CODE', 'PRODUCT_NUMBER': 'SALES_TARGETDATA_PRODUCT_NUMBER', 'RETAILER_CODE': 'SALES_TARGETDATA_RETAILER_CODE'})

all_dataframes.append(sales_target_data)

sales_target_data.head()

,Id,SALES_TARGETDATA_SALES_STAFF_CODE,SALES_TARGETDATA_SALES_YEAR,SALES_TARGETDATA_SALES_PERIOD,SALES_TARGETDATA_RETAILER_NAME,SALES_TARGETDATA_SALES_TARGET,TRIAL888_product,PRODUCT_number,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCT_TYPE_code,...,PRODUCT_description_description,TRIAL888_x,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code_en,TRIAL888_y,PRODUCT_PRODUCT_LINE_code_en,TRIAL888_retailer,RETAILER_code,TRIAL219,TRIAL222
0,1,6,2021,5,Altitudes extrêmes,489,T,53,10-1-2021,8,...,This climbing lamp is water-resistant. It come...,T,2,Climbing Accessories,T,Mountaineering Equipment,T,33,T,T
1,2,6,2021,5,Altitudes extrêmes,666,T,55,10-1-2021,8,...,Rechargeable battery pack for the Firefly Clim...,T,2,Climbing Accessories,T,Mountaineering Equipment,T,33,T,T
2,3,6,2021,5,Altitudes extrêmes,682,T,56,10-1-2021,8,...,Nylon web bag perfect for holding several stic...,T,2,Climbing Accessories,T,Mountaineering Equipment,T,33,T,T
3,4,6,2021,5,Camping Sauvage,3193,T,76,15-2-2011,12,...,Knife is 33 cm long with a 20 cm blade made of...,T,3,Knives,T,Personal Accessories,T,35,T,T
4,5,6,2021,5,Camping Sauvage,7236,T,85,12-6-2015,14,...,Hand held GPS receiver with color display. Inc...,T,3,Navigation,T,Personal Accessories,T,35,T,T


### Retailer_segment

In [32]:
retailer_segment = go_crm_retailer_segment

# rename the columns
retailer_segment = retailer_segment.rename(columns={'SEGMENT_CODE': 'RETAILER_SEGMENT_segment_code', 'LANGUAGE': 'RETAILER_SEGMENT_language', 'SEGMENT_NAME': 'RETAILER_SEGMENT_segment_name', 'SEGMENT_DESCRIPTION': 'RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'})

all_dataframes.append(retailer_segment)

retailer_segment.head()

,RETAILER_SEGMENT_segment_code,RETAILER_SEGMENT_language,RETAILER_SEGMENT_segment_name,RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description,TRIAL222
0,1,EN,Outdoors Shop - Exclusive,"Primarily offers camping supplies, hiking equi...",T
1,10,EN,Eyewear Store,Primarily offers prescription and fashion eyew...,T
2,11,EN,Sports Store - Exclusive,Primarily offers a wide range of sporting good...,T
3,12,EN,Sports Store,Primarily offers a wide range of sporting goods.,T
4,2,EN,Outdoors Shop,"Primarily offers camping supplies, hiking equi...",T


### Retailer_headquarters

In [33]:
retailer_headquarters = go_crm_retailer_headquarters

# rename the columns
retailer_headquarters = retailer_headquarters.rename(columns={'RETAILER_CODEMR': 'RETAILER_HEADQUARTER_codemr', 'RETAILER_NAME' : 'RETAIL_HEADQUARTER_retailer_name', 'ADDRESS1' : 'RETAILER_HEADQUARTER_address1_address', 'ADDRESS2' : 'RETAILER_HEADQUARTER_address2_address', 'CITY' : 'RETAILER_HEADQUARTER_city_city', 'REGION' : 'RETAILER_HEADQUARTER_region_region', 'POSTAL_ZONE' : 'RETAILER_HEADQUARTER_postal_zone_postal_zone', 'COUNTRY_CODE' : 'RETAILER_HEADQUARTER_country_code_country', 'PHONE' : 'RETAILER_HEADQUARTER_phone_phone', 'FAX': 'RETAILER_HEADQUARTER_fax_fax', 'SEGMENT_CODE' : 'RETAILER_HEADQUARTER_segment_code'})

all_dataframes.append(retailer_headquarters)

retailer_headquarters.head()

,RETAILER_HEADQUARTER_codemr,RETAIL_HEADQUARTER_retailer_name,RETAILER_HEADQUARTER_address1_address,RETAILER_HEADQUARTER_address2_address,RETAILER_HEADQUARTER_city_city,RETAILER_HEADQUARTER_region_region,RETAILER_HEADQUARTER_postal_zone_postal_zone,RETAILER_HEADQUARTER_country_code_country,RETAILER_HEADQUARTER_phone_phone,RETAILER_HEADQUARTER_fax_fax,RETAILER_HEADQUARTER_segment_code,TRIAL222
0,1,Rock Steady,"45, rue Atwater",None,Montréal,Québec,H2T 9K8,4,1 (514) 747-4481,1 (514) 741-2277,2,T
1,10,Excellence en Montagne,"160, rue Moureaux",Bureau 410,Ville de Québec,Québec,G1B 3L5,4,1 (418) 653-6456,1 (418) 653-2311,2,T
2,11,Vacation Central,"3000, 58 Merivale Road",None,Ottawa,Ontario,K0T 9R0,4,1 (613) 736-6443,1 (613) 736-5593,6,T
3,12,Outdoor Central Ltd,"Suite 23, 180 McKercher Drive",None,Regina,Saskatchewan,S6J 3G0,4,1 (306) 243-8764,1 (306) 243-5213,2,T
4,13,Precipice Equipment,23 Blackcomb Way,None,Whistler,British Columbia,V6T 1Y8,4,1 (604) 938-7564,1 (604) 938-2324,1,T


### Verwijderen trial kolommen

In [34]:
for df in all_dataframes:
    trial_columns = df.filter(like='TRIAL').columns
    df.drop(columns=trial_columns, inplace=True)

# Omzetten van de data
Omzetten van de data naar de juiste data types.
Year en Date zijn al omgezet naar de juiste data types.

### Sales_demographic

In [35]:
sales_demographic['SALES_DEMOGRAPHIC_code'] = sales_demographic['SALES_DEMOGRAPHIC_code'].astype('int')
sales_demographic['SALES_DEMOGRAPHIC_AGE_GROUP_CODE'] = sales_demographic['SALES_DEMOGRAPHIC_AGE_GROUP_CODE'].astype('int')
sales_demographic['SALES_DEMOGRAPHIC_RETAILER_CODE'] = sales_demographic['SALES_DEMOGRAPHIC_RETAILER_CODE'].astype('int')
sales_demographic['SALES_DEMOGRAPHIC_SALES_PERCENT'] = sales_demographic['SALES_DEMOGRAPHIC_SALES_PERCENT'].astype('int')
sales_demographic['SALES_DEMOGRAPHIC_UPPER_AGE'] = sales_demographic['SALES_DEMOGRAPHIC_UPPER_AGE'].astype('int')
sales_demographic['SALES_DEMOGRAPHIC_LOWER_AGE'] = sales_demographic['SALES_DEMOGRAPHIC_LOWER_AGE'].astype('int')

sales_demographic.dtypes

SALES_DEMOGRAPHIC_code              int32
SALES_DEMOGRAPHIC_RETAILER_CODE     int32
SALES_DEMOGRAPHIC_AGE_GROUP_CODE    int32
SALES_DEMOGRAPHIC_SALES_PERCENT     int32
SALES_DEMOGRAPHIC_UPPER_AGE         int32
SALES_DEMOGRAPHIC_LOWER_AGE         int32
dtype: object

### Unit

In [36]:
unit['UNIT_COST_cost'] = unit['UNIT_COST_cost'].astype('float64')
unit['UNIT_PRICE_price'] = unit['UNIT_PRICE_price'].astype('float64')
unit['UNIT_SALE_sale'] = unit['UNIT_SALE_sale'].astype('float64')

unit.dtypes

UNIT_COST_cost       float64
UNIT_PRICE_price     float64
UNIT_SALE_sale       float64
UNIT_id                int64
ORDER_DETAIL_CODE     object
dtype: object

### Sales_staff

In [37]:
sales_staff['SALES_STAFF_code'] = sales_staff['SALES_STAFF_code'].astype('int')
sales_staff['SALES_STAFF_email'] = sales_staff['SALES_STAFF_email'].astype('string').replace({np.nan: None})
sales_staff['SALES_STAFF_extension'] = sales_staff['SALES_STAFF_extension'].astype('string').replace({np.nan: None})

sales_staff['SALES_STAFF_POSITION_EN_position'] = sales_staff['SALES_STAFF_POSITION_EN_position'].astype('string').replace({np.nan: None})
sales_staff['SALES_STAFF_WORK_PHONE_work_phone'] = sales_staff['SALES_STAFF_WORK_PHONE_work_phone'].astype('string').replace({np.nan: None})
sales_staff['SALES_STAFF_DATE_HIRED_hired'] = pd.to_datetime(sales_staff['SALES_STAFF_DATE_HIRED_hired'], format='mixed')
sales_staff['SALES_STAFF_MANAGER_CODE_manager'] = sales_staff['SALES_STAFF_MANAGER_CODE_manager'].astype('int64')
sales_staff['SALES_STAFF_FAX'] = sales_staff['SALES_STAFF_FAX'].astype('string').replace({np.nan: None})
sales_staff['SALES_STAFF_FIRST_NAME_first_name'] = sales_staff['SALES_STAFF_FIRST_NAME_first_name'].astype('string').replace({np.nan: None})
sales_staff['SALES_STAFF_LAST_NAME_last_name'] = sales_staff['SALES_STAFF_LAST_NAME_last_name'].astype('string').replace({np.nan: None})
sales_staff['SALES_STAFF_SALES_BRANCH_CODE_branch_code'] = sales_staff['SALES_STAFF_SALES_BRANCH_CODE_branch_code'].astype('int64')
sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS1_address'] = sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS1_address'].astype('string').replace({np.nan: None})
sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS2_address'] = sales_staff['SALES_STAFF_SALES_BRANCH_ADDRESS2_address'].astype('string').replace({np.nan: None})


sales_staff.head()

,SALES_STAFF_code,SALES_STAFF_email,SALES_STAFF_extension,SALES_STAFF_POSITION_EN_position,SALES_STAFF_WORK_PHONE_work_phone,SALES_STAFF_DATE_HIRED_hired,SALES_STAFF_MANAGER_CODE_manager,SALES_STAFF_FAX,SALES_STAFF_FIRST_NAME_first_name,SALES_STAFF_LAST_NAME_last_name,SALES_STAFF_SALES_BRANCH_ADDRESS1_address,SALES_STAFF_SALES_BRANCH_ADDRESS2_address,SALES_STAFF_SALES_BRANCH_CODE_branch_code
0,100,TSavolainen@grtd123.com,825,Level 2 Sales Representative,+358(0)17 - 433 127,1998-07-23,18,+358(0)17 - 433 129,Tuomas,Savolainen,Kauppakatu 33,None,31
1,101,CKim@grtd123.com,1228,Level 2 Sales Representative,82-2-778-6587,1998-12-15,87,82-2-778-6586,Chang-ho,Kim,"234-12, Kongdeok-Dong",Mapo-Ku,32
2,102,JChoi@grtd123.com,2641,Level 3 Sales Representative,82-2-778-6587,1998-11-27,87,82-2-778-6586,Jung-ho,Choi,"234-12, Kongdeok-Dong",Mapo-Ku,32
3,103,KKwao@grtd123.com,1541,Level 2 Sales Representative,886-2-2975-1988,1998-10-07,87,886-2-2989-4978,Kwei-Ping,Kao,"154, Choung-Cheng South Road",None,33
4,104,CFon@grtd123.com,2311,Level 3 Sales Representative,886-2-2975-1988,1996-05-08,87,886-2-2989-4978,Chin-Tsai,Fang,"154, Choung-Cheng South Road",None,33


### satisfaction_type

In [38]:
satisfaction_type['SATISFACTION_TYPE_code'] = satisfaction_type['SATISFACTION_TYPE_code'].astype('int')
satisfaction_type['SATISFACTION_TYPE_DESCRIPTION'] = satisfaction_type['SATISFACTION_TYPE_DESCRIPTION'].astype('string').replace({np.nan: None})

satisfaction_type.dtypes

SATISFACTION_TYPE_code                    int32
SATISFACTION_TYPE_DESCRIPTION    string[python]
dtype: object

### Course

In [39]:
course['COURSE_code'] = course['COURSE_code'].astype('int')
course['COURSE_DESCRIPTION'] = course['COURSE_DESCRIPTION'].astype('string').replace({np.nan: None})

course.dtypes

COURSE_code                    int32
COURSE_DESCRIPTION    string[python]
dtype: object

### Order

In [40]:
order['ORDER_ORDER_METHOD_CODE_method_code'] = order['ORDER_ORDER_METHOD_CODE_method_code'].astype('int')
order['ORDER_order_number'] = order['ORDER_order_number'].astype('int')
order['ORDER_ORDER_METHOD_EN_method'] = order['ORDER_ORDER_METHOD_EN_method'].astype('string').replace({np.nan: None})

order.dtypes

ORDER_ORDER_METHOD_CODE_method_code             int32
ORDER_order_number                              int32
ORDER_ORDER_METHOD_EN_method           string[python]
dtype: object

### Retailer_site

In [41]:
retailer_site['RETAILER_SITE_code'] = retailer_site['RETAILER_SITE_code'].astype('int')
retailer_site['RETAILER_SITE_COUNTRY_CODE_country'] = retailer_site['RETAILER_SITE_COUNTRY_CODE_country'].astype('string').replace({np.nan: None})
retailer_site['RETAILER_SITE_CITY_city'] = retailer_site['RETAILER_SITE_CITY_city'].astype('string').replace({np.nan: None})
retailer_site['RETAILER_SITE_REGION_region'] = retailer_site['RETAILER_SITE_REGION_region'].astype('string').replace({np.nan: None})
retailer_site['RETAILER_SITE_POSTAL_ZONE_postal_zone'] = retailer_site['RETAILER_SITE_POSTAL_ZONE_postal_zone'].astype('string').replace({np.nan: None})
retailer_site['RETAILER_SITE_RETAILER_CODE_retailer_code'] = retailer_site['RETAILER_SITE_RETAILER_CODE_retailer_code'].astype('int')
retailer_site['RETAILER_SITE_ACTIVE_INDICATOR_indicator'] = retailer_site['RETAILER_SITE_ACTIVE_INDICATOR_indicator'].astype('int')
retailer_site['RETAILER_SITE_ADDRESS1_address'] = retailer_site['RETAILER_SITE_ADDRESS1_address'].astype('string').replace({np.nan: None})
retailer_site['RETAILER_SITE_ADDRESS2_address'] = retailer_site['RETAILER_SITE_ADDRESS2_address'].astype('string').replace({np.nan: None})


retailer_site.dtypes

RETAILER_SITE_code                                    int32
RETAILER_SITE_RETAILER_CODE_retailer_code             int32
RETAILER_SITE_ADDRESS1_address               string[python]
RETAILER_SITE_ADDRESS2_address                       object
RETAILER_SITE_CITY_city                      string[python]
RETAILER_SITE_REGION_region                          object
RETAILER_SITE_POSTAL_ZONE_postal_zone                object
RETAILER_SITE_COUNTRY_CODE_country           string[python]
RETAILER_SITE_ACTIVE_INDICATOR_indicator              int32
dtype: object

### Sales_branch

In [42]:
sales_branch['SALES_BRANCH_code'] = sales_branch['SALES_BRANCH_code'].astype('int')
sales_branch['SALES_BRANCH_COUNTRY_CODE_country'] = sales_branch['SALES_BRANCH_COUNTRY_CODE_country'].astype('string').replace({np.nan: None})
sales_branch['SALES_BRANCH_REGION_region'] = sales_branch['SALES_BRANCH_REGION_region'].astype('string').replace({np.nan: None})
sales_branch['SALES_BRANCH_CITY_city'] = sales_branch['SALES_BRANCH_CITY_city'].astype('string').replace({np.nan: None})
sales_branch['SALES_BRANCH_POSTAL_ZONE_postal_zone'] = sales_branch['SALES_BRANCH_POSTAL_ZONE_postal_zone'].astype('string').replace({np.nan: None})
sales_branch['SALES_BRANCH_ADDRESS1_address'] = sales_branch['SALES_BRANCH_ADDRESS1_address'].astype('string').replace({np.nan: None})
sales_branch['SALES_BRANCH_ADDRESS2_address'] = sales_branch['SALES_BRANCH_ADDRESS2_address'].astype('string').replace({np.nan: None})

sales_branch.dtypes

SALES_BRANCH_code                                int32
SALES_BRANCH_ADDRESS1_address           string[python]
SALES_BRANCH_ADDRESS2_address                   object
SALES_BRANCH_CITY_city                  string[python]
SALES_BRANCH_REGION_region                      object
SALES_BRANCH_POSTAL_ZONE_postal_zone    string[python]
SALES_BRANCH_COUNTRY_CODE_country       string[python]
dtype: object

### Retailer_contact

In [43]:
retailer_contact['RETAILER_CONTACT_code'] = retailer_contact['RETAILER_CONTACT_code'].astype('int')
retailer_contact['RETAILER_CONTACT_email'] = retailer_contact['RETAILER_CONTACT_email'].astype('string').replace({np.nan: None})
retailer_contact['RETAILER_CONTACT_RETAILER_SITE_CODE_site_code'] = retailer_contact['RETAILER_CONTACT_RETAILER_SITE_CODE_site_code'].astype('int')
retailer_contact['RETAILER_CONTACT_JOB_POSITION_EN_position'] = retailer_contact['RETAILER_CONTACT_JOB_POSITION_EN_position'].astype('string').replace({np.nan: None})
retailer_contact['RETAILER_CONTACT_EXTENSION_extension'] = retailer_contact['RETAILER_CONTACT_EXTENSION_extension'].astype('string').replace({np.nan: None})
retailer_contact['RETAILER_CONTACT_FAX_fax'] = retailer_contact['RETAILER_CONTACT_FAX_fax'].astype('string').replace({np.nan: None})
retailer_contact['RETAILER_CONTACT_GENDER_gender'] = retailer_contact['RETAILER_CONTACT_GENDER_gender'].astype('string').replace({np.nan: None})
retailer_contact['RETAILER_CONTACT_FIRST_NAME_first_name'] = retailer_contact['RETAILER_CONTACT_FIRST_NAME_first_name'].astype('string').replace({np.nan: None})
retailer_contact['RETAILER_CONTACT_LAST_NAME_last_name'] = retailer_contact['RETAILER_CONTACT_LAST_NAME_last_name'].astype('string').replace({np.nan: None})

retailer_contact.dtypes

RETAILER_CONTACT_code                                     int32
RETAILER_CONTACT_RETAILER_SITE_CODE_site_code             int32
RETAILER_CONTACT_FIRST_NAME_first_name           string[python]
RETAILER_CONTACT_LAST_NAME_last_name             string[python]
RETAILER_CONTACT_JOB_POSITION_EN_position        string[python]
RETAILER_CONTACT_EXTENSION_extension                     object
RETAILER_CONTACT_FAX_fax                                 object
RETAILER_CONTACT_email                                   object
RETAILER_CONTACT_GENDER_gender                   string[python]
dtype: object

### Retailer

In [44]:
retailer['RETAILER_code'] = retailer['RETAILER_code'].astype('int')
retailer['RETAILER_name'] = retailer['RETAILER_name'].astype('string').replace({np.nan: None})
retailer['RETAILER_COMPANY_CODE_MR_company'] = retailer['RETAILER_COMPANY_CODE_MR_company'].astype('string').replace({np.nan: None})
retailer['RETAILER_RETAILER_TYPE_code'] = retailer['RETAILER_RETAILER_TYPE_code'].astype('int')
retailer['RETAILER_RETAILER_TYPE_EN'] = retailer['RETAILER_RETAILER_TYPE_EN'].astype('string').replace({np.nan: None})

retailer.dtypes

RETAILER_code                                int32
RETAILER_COMPANY_CODE_MR_company            object
RETAILER_name                       string[python]
RETAILER_RETAILER_TYPE_code                  int32
RETAILER_RETAILER_TYPE_EN           string[python]
dtype: object

### Product

In [45]:
product['PRODUCT_number'] = product['PRODUCT_number'].astype('int')
product['PRODUCT_name_product'] = product['PRODUCT_name_product'].astype('string').replace({np.nan: None})
product['PRODUCT_description_description'] = product['PRODUCT_description_description'].astype('string').replace({np.nan: None})
product['PRODUCT_image_image'] = product['PRODUCT_image_image'].astype('string').replace({np.nan: None})
product['PRODUCT_INTRODUCTION_DATE_introduced'] = pd.to_datetime(product['PRODUCT_INTRODUCTION_DATE_introduced'], format='mixed')
product['PRODUCT_PRODUCTION_COST_cost'] = product['PRODUCT_PRODUCTION_COST_cost'].astype('float')
product['PRODUCT_MARGIN_margin'] = product['PRODUCT_MARGIN_margin'].astype('float')
product['PRODUCT_LANGUAGE_language'] = product['PRODUCT_LANGUAGE_language'].astype('string').replace({np.nan: None})
product['PRODUCT_PRODUCT_LINE_code'] = product['PRODUCT_PRODUCT_LINE_code'].astype('string').replace({np.nan: None})
product['PRODUCT_PRODUCT_LINE_code_en'] = product['PRODUCT_PRODUCT_LINE_code_en'].astype('string').replace({np.nan: None})
product['PRODUCT_PRODUCT_TYPE_code'] = product['PRODUCT_PRODUCT_TYPE_code'].astype('string').replace({np.nan: None})
product['PRODUCT_PRODUCT_TYPE_code_en'] = product['PRODUCT_PRODUCT_TYPE_code_en'].astype('string').replace({np.nan: None})

product.dtypes

PRODUCT_number                                   int32
PRODUCT_INTRODUCTION_DATE_introduced    datetime64[ns]
PRODUCT_PRODUCT_TYPE_code               string[python]
PRODUCT_PRODUCTION_COST_cost                   float64
PRODUCT_MARGIN_margin                          float64
PRODUCT_image_image                     string[python]
PRODUCT_LANGUAGE_language               string[python]
PRODUCT_name_product                    string[python]
PRODUCT_description_description         string[python]
PRODUCT_PRODUCT_LINE_code               string[python]
PRODUCT_PRODUCT_TYPE_code_en            string[python]
PRODUCT_PRODUCT_LINE_code_en            string[python]
dtype: object

### Order_details

In [46]:
order_details['ORDER_DETAILS_QUANTITY_quantity'] = order_details['ORDER_DETAILS_QUANTITY_quantity'].astype('int')
order_details['ORDER_DETAILS_ORDER_NUMBER_order'] = order_details['ORDER_DETAILS_ORDER_NUMBER_order'].astype('int')
order_details['ORDER_DETAILS_PRODUCT_NUMBER_product'] = order_details['ORDER_DETAILS_PRODUCT_NUMBER_product'].astype('int')
order_details['UNIT_COST_cost'] = order_details['UNIT_COST_cost'].astype('float64')
order_details['UNIT_PRICE_price'] = order_details['UNIT_PRICE_price'].astype('float64')
order_details['UNIT_SALE_sale'] = order_details['UNIT_SALE_sale'].astype('float64')
order_details['ORDER_DETAIL_code'] = order_details['ORDER_DETAIL_code'].astype('int')

# unit is already converted
order_details.dtypes

ORDER_DETAIL_code                         int32
ORDER_DETAILS_ORDER_NUMBER_order          int32
ORDER_DETAILS_PRODUCT_NUMBER_product      int32
ORDER_DETAILS_QUANTITY_quantity           int32
UNIT_id                                   int64
UNIT_COST_cost                          float64
UNIT_PRICE_price                        float64
UNIT_SALE_sale                          float64
dtype: object

### Returned_item

In [47]:
returned_item['RETURNED_ITEM_code'] = returned_item['RETURNED_ITEM_code'].astype('int')
returned_item['RETURNED_ITEM_DATE'] = pd.to_datetime(returned_item['RETURNED_ITEM_DATE'], format='mixed')
returned_item['RETURNED_ITEM_QUANTITY'] = returned_item['RETURNED_ITEM_QUANTITY'].astype('int')
returned_item['RETURNED_ITEM_ORDER_DETAIL_CODE'] = returned_item['RETURNED_ITEM_ORDER_DETAIL_CODE'].astype('int')
returned_item['RETURNED_ITEM_RETURN_REASON_code'] = returned_item['RETURNED_ITEM_RETURN_REASON_code'].astype('int')
returned_item['RETURNED_ITEM_RETURN_REASON_description_en'] = returned_item['RETURNED_ITEM_RETURN_REASON_description_en'].astype('string').replace({np.nan: None})

returned_item.dtypes

RETURNED_ITEM_code                                     int32
RETURNED_ITEM_DATE                            datetime64[ns]
RETURNED_ITEM_QUANTITY                                 int32
RETURNED_ITEM_ORDER_DETAIL_CODE                        int32
RETURNED_ITEM_RETURN_REASON_code                       int32
RETURNED_ITEM_RETURN_REASON_description_en    string[python]
dtype: object

### Sales_Targetdata

In [48]:
sales_target_data['SALES_TARGETDATA_SALES_YEAR'] = sales_target_data['SALES_TARGETDATA_SALES_YEAR'].astype('int')
sales_target_data['SALES_TARGETDATA_SALES_PERIOD'] = sales_target_data['SALES_TARGETDATA_SALES_PERIOD'].astype('string').replace({np.nan: None})
sales_target_data['SALES_TARGETDATA_RETAILER_NAME'] = sales_target_data['SALES_TARGETDATA_RETAILER_NAME'].astype('string').replace({np.nan: None})
sales_target_data['SALES_TARGETDATA_SALES_TARGET'] = sales_target_data['SALES_TARGETDATA_SALES_TARGET'].astype('float64')
sales_target_data['SALES_TARGETDATA_SALES_STAFF_CODE'] = sales_target_data['SALES_TARGETDATA_SALES_STAFF_CODE'].astype('int')
sales_target_data['PRODUCT_number'] = sales_target_data['PRODUCT_number'].astype('int')
sales_target_data['PRODUCT_INTRODUCTION_DATE_introduced'] = pd.to_datetime(sales_target_data['PRODUCT_INTRODUCTION_DATE_introduced'], format='mixed')
sales_target_data['PRODUCT_PRODUCTION_COST_cost'] = sales_target_data['PRODUCT_PRODUCTION_COST_cost'].astype('float64')
sales_target_data['PRODUCT_PRODUCT_TYPE_code'] = sales_target_data['PRODUCT_PRODUCT_TYPE_code'].astype('string').replace({np.nan: None})
sales_target_data['PRODUCT_PRODUCT_TYPE_code_en'] = sales_target_data['PRODUCT_PRODUCT_TYPE_code_en'].astype('string').replace({np.nan: None})
sales_target_data['PRODUCT_PRODUCT_LINE_code'] = sales_target_data['PRODUCT_PRODUCT_LINE_code'].astype('string').replace({np.nan: None})
sales_target_data['PRODUCT_PRODUCT_LINE_code_en'] = sales_target_data['PRODUCT_PRODUCT_LINE_code_en'].astype('string').replace({np.nan: None})
sales_target_data['RETAILER_code'] = sales_target_data['RETAILER_code'].astype('int')
sales_target_data['PRODUCT_MARGIN_margin'] = sales_target_data['PRODUCT_MARGIN_margin'].astype('float64')
sales_target_data['PRODUCT_image_image'] = sales_target_data['PRODUCT_image_image'].astype('string').replace({np.nan: None})
sales_target_data['PRODUCT_LANGUAGE_language'] = sales_target_data['PRODUCT_LANGUAGE_language'].astype('string').replace({np.nan: None})
sales_target_data['PRODUCT_name_product'] = sales_target_data['PRODUCT_name_product'].astype('string').replace({np.nan: None})
sales_target_data['PRODUCT_description_description'] = sales_target_data['PRODUCT_description_description'].astype('string').replace({np.nan: None})

sales_target_data.dtypes

Id                                               int64
SALES_TARGETDATA_SALES_STAFF_CODE                int32
SALES_TARGETDATA_SALES_YEAR                      int32
SALES_TARGETDATA_SALES_PERIOD           string[python]
SALES_TARGETDATA_RETAILER_NAME          string[python]
SALES_TARGETDATA_SALES_TARGET                  float64
PRODUCT_number                                   int32
PRODUCT_INTRODUCTION_DATE_introduced    datetime64[ns]
PRODUCT_PRODUCT_TYPE_code               string[python]
PRODUCT_PRODUCTION_COST_cost                   float64
PRODUCT_MARGIN_margin                          float64
PRODUCT_image_image                     string[python]
PRODUCT_LANGUAGE_language               string[python]
PRODUCT_name_product                    string[python]
PRODUCT_description_description         string[python]
PRODUCT_PRODUCT_LINE_code               string[python]
PRODUCT_PRODUCT_TYPE_code_en            string[python]
PRODUCT_PRODUCT_LINE_code_en            string[python]
RETAILER_c

### Training

In [49]:
training['TRAINING_SALES_STAFF_CODE'] = training['TRAINING_SALES_STAFF_CODE'].astype('int')
training['TRAINING_COURSE_CODE'] = training['TRAINING_COURSE_CODE'].astype('int')
training['TRAINING_YEAR'] = training['TRAINING_YEAR'].astype('int')
training['COURSE_code'] = training['COURSE_code'].astype('int')
training['COURSE_DESCRIPTION'] = training['COURSE_DESCRIPTION'].astype('string').replace({np.nan: None})

training.dtypes

TRAINING_YEAR                         int32
TRAINING_SALES_STAFF_CODE             int32
TRAINING_COURSE_CODE                  int32
COURSE_code                           int32
COURSE_DESCRIPTION           string[python]
dtype: object

### Satisfaction

In [50]:
satisfaction['SATISFACTION_SALES_STAFF_CODE'] = satisfaction['SATISFACTION_SALES_STAFF_CODE'].astype('int')
satisfaction['SATISFACTION_SATISFACTION_TYPE_CODE'] = satisfaction['SATISFACTION_SATISFACTION_TYPE_CODE'].astype('int')
satisfaction['SATISFACTION_YEAR'] = satisfaction['SATISFACTION_YEAR'].astype('int')
satisfaction['SATISFACTION_TYPE_code'] = satisfaction['SATISFACTION_TYPE_code'].astype('int')
satisfaction['SATISFACTION_TYPE_DESCRIPTION'] = satisfaction['SATISFACTION_TYPE_DESCRIPTION'].astype('string').replace({np.nan: None})

satisfaction.dtypes

SATISFACTION_YEAR                               int32
SATISFACTION_SALES_STAFF_CODE                   int32
SATISFACTION_SATISFACTION_TYPE_CODE             int32
SATISFACTION_TYPE_code                          int32
SATISFACTION_TYPE_DESCRIPTION          string[python]
dtype: object

### Order_header

In [51]:
order_header['ORDER_HEADER_number'] = order_header['ORDER_HEADER_number'].astype('int')
order_header['ORDER_HEADER_SALES_STAFF_CODE'] = order_header['ORDER_HEADER_SALES_STAFF_CODE'].astype('int')
order_header['ORDER_HEADER_SALES_BRANCH_CODE'] = order_header['ORDER_HEADER_SALES_BRANCH_CODE'].astype('int')
order_header['ORDER_HEADER_RETAILER_SITE_CODE'] = order_header['ORDER_HEADER_RETAILER_SITE_CODE'].astype('int')
order_header['ORDER_HEADER_RETAILER_CONTACT_CODE'] = order_header['ORDER_HEADER_RETAILER_CONTACT_CODE'].astype('int')
order_header['ORDER_METHOD_CODE'] = order_header['ORDER_METHOD_CODE'].astype('int')
order_header['RETAILER_CODE'] = order_header['RETAILER_code'].astype('int')
order_header['RETAILER_COMPANY_CODE_MR_company'] = order_header['RETAILER_COMPANY_CODE_MR_company'].astype('string').replace({np.nan: None})
order_header['RETAILER_NAME'] = order_header['RETAILER_name'].astype('string').replace({np.nan: None})

order_header.dtypes

ORDER_HEADER_number                                       int32
RETAILER_NAME                                    string[python]
ORDER_HEADER_RETAILER_SITE_CODE                           int32
ORDER_HEADER_RETAILER_CONTACT_CODE                        int32
ORDER_HEADER_SALES_STAFF_CODE                             int32
ORDER_HEADER_SALES_BRANCH_CODE                            int32
ORDER_HEADER_ORDER_DATE                          datetime64[ns]
ORDER_METHOD_CODE                                         int32
RETAILER_code                                            object
RETAILER_COMPANY_CODE_MR_company                         object
RETAILER_name                                            object
RETAILER_RETAILER_TYPE_code                              object
RETAILER_RETAILER_TYPE_EN                                object
SALES_STAFF_code                                         object
SALES_STAFF_email                                        object
SALES_STAFF_extension                   

### Retailer_segment

In [52]:
retailer_segment['RETAILER_SEGMENT_segment_code'] = retailer_segment['RETAILER_SEGMENT_segment_code'].astype('int')
retailer_segment['RETAILER_SEGMENT_language'] = retailer_segment['RETAILER_SEGMENT_language'].astype('string').replace({np.nan: None})
retailer_segment['RETAILER_SEGMENT_segment_name'] = retailer_segment['RETAILER_SEGMENT_segment_name'].astype('string').replace({np.nan: None})
retailer_segment['RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'] = retailer_segment['RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'].astype('string').replace({np.nan: None})

retailer_segment.dtypes

RETAILER_SEGMENT_segment_code                                int32
RETAILER_SEGMENT_language                           string[python]
RETAILER_SEGMENT_segment_name                       string[python]
RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description    string[python]
dtype: object

### Retailer_headquarters

In [53]:
retailer_headquarters['RETAILER_HEADQUARTER_codemr'] = retailer_headquarters['RETAILER_HEADQUARTER_codemr'].astype('int')
retailer_headquarters['RETAIL_HEADQUARTER_retailer_name'] = retailer_headquarters['RETAIL_HEADQUARTER_retailer_name'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_address1_address'] = retailer_headquarters['RETAILER_HEADQUARTER_address1_address'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_address2_address'] = retailer_headquarters['RETAILER_HEADQUARTER_address2_address'].astype('string').replace({np.nan: None})

retailer_headquarters['RETAILER_HEADQUARTER_city_city'] = retailer_headquarters['RETAILER_HEADQUARTER_city_city'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_region_region'] = retailer_headquarters['RETAILER_HEADQUARTER_region_region'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_postal_zone_postal_zone'] = retailer_headquarters['RETAILER_HEADQUARTER_postal_zone_postal_zone'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_country_code_country'] = retailer_headquarters['RETAILER_HEADQUARTER_country_code_country'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_phone_phone'] = retailer_headquarters['RETAILER_HEADQUARTER_phone_phone'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_fax_fax'] = retailer_headquarters['RETAILER_HEADQUARTER_fax_fax'].astype('string').replace({np.nan: None})
retailer_headquarters['RETAILER_HEADQUARTER_segment_code'] = retailer_headquarters['RETAILER_HEADQUARTER_segment_code'].astype('int')

retailer_headquarters

#retailer_headquarters.dtypes

,RETAILER_HEADQUARTER_codemr,RETAIL_HEADQUARTER_retailer_name,RETAILER_HEADQUARTER_address1_address,RETAILER_HEADQUARTER_address2_address,RETAILER_HEADQUARTER_city_city,RETAILER_HEADQUARTER_region_region,RETAILER_HEADQUARTER_postal_zone_postal_zone,RETAILER_HEADQUARTER_country_code_country,RETAILER_HEADQUARTER_phone_phone,RETAILER_HEADQUARTER_fax_fax,RETAILER_HEADQUARTER_segment_code
0,1,Rock Steady,"45, rue Atwater",None,Montréal,Québec,H2T 9K8,4,1 (514) 747-4481,1 (514) 741-2277,2
1,10,Excellence en Montagne,"160, rue Moureaux",Bureau 410,Ville de Québec,Québec,G1B 3L5,4,1 (418) 653-6456,1 (418) 653-2311,2
2,11,Vacation Central,"3000, 58 Merivale Road",None,Ottawa,Ontario,K0T 9R0,4,1 (613) 736-6443,1 (613) 736-5593,6
3,12,Outdoor Central Ltd,"Suite 23, 180 McKercher Drive",None,Regina,Saskatchewan,S6J 3G0,4,1 (306) 243-8764,1 (306) 243-5213,2
4,13,Precipice Equipment,23 Blackcomb Way,None,Whistler,British Columbia,V6T 1Y8,4,1 (604) 938-7564,1 (604) 938-2324,1
...,...,...,...,...,...,...,...,...,...,...,...
409,9,Over the Top Cycles,23 Nowlan Drive,None,Moncton,New Brunswick,E1A 2A7,4,1 (506) 849-8756,1 (506) 849-6432,7
410,90,Weston Outfitters,738 Greyling Street,None,San Diego,California,92129,3,1 (619) 223-1235,1 (619) 223-1245,2
411,91,"Artículos de Campismo El Aquila, S.A. de C.V.",Avda. Chapultepec 246-303,Virreyes,Distrito Federal,México,06700,20,+(525) 684 7127,+(525) 684 7119,2
412,92,"Todo para el Golf, S.A. de C.V.",Francisco Munguía No. 45,Atahualpa,Monterey,Nuevo León,30530,20,+(528) 366 6773,+(528) 366 6717,3


# Toevoegen van nieuwe kolommen
Toevoegen van de afgeleide informatie die ook in het ETL en de data warehouse moet komen.
Full name is al gedaan via sql

In [54]:
sales_target_data['SALES_TARGET_DATA_TARGET_COST'] = sales_target_data['SALES_TARGETDATA_SALES_TARGET'] * sales_target_data['PRODUCT_PRODUCTION_COST_cost']

sales_target_data['SALES_TARGET_DATA_TARGET_MARGIN'] = sales_target_data['SALES_TARGETDATA_SALES_TARGET'] * sales_target_data['PRODUCT_MARGIN_margin']

order_details['ORDER_DETAILS_TOTAL_COST_total'] = order_details['ORDER_DETAILS_QUANTITY_quantity'] * order_details['UNIT_COST_cost']

order_details['ORDER_DETAILS_TOTAL_MARGIN_margin'] = order_details['ORDER_DETAILS_QUANTITY_quantity'] * order_details['UNIT_SALE_sale']

sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_COST'] = sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME'].astype('int') * sales_product_forecast['PRODUCT_PRODUCTION_COST_cost'].astype('float')
sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN'] = sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME'].astype('int') * sales_product_forcast['PRODUCT_MARGIN_margin'].astype('float')

# Dimensies
returned_item['RETURNED_ITEM_RETURNED_ITEMS_TOTAL_PRICE'] = returned_item['RETURNED_ITEM_QUANTITY'] * order_details['UNIT_SALE_sale']
course['COURSE_DESCRIPTION_SHORT'] = None
retailer_segment['SEGMENT_DESCRIPTION_SHORT'] = None
satisfaction_type['SATISFACTION_TYPE_DESCRIPTION_SHORT'] = None

# if address2 is empty, main_address should be address1 and vice versa. If both addresses are available, main_address should be null
sales_branch['MAIN_ADDRESS'] = np.where(sales_branch['SALES_BRANCH_ADDRESS2_address'].isna(), sales_branch['SALES_BRANCH_ADDRESS1_address'], np.where(sales_branch['SALES_BRANCH_ADDRESS1_address'].isna(), sales_branch['SALES_BRANCH_ADDRESS2_address'], None))

retailer_site['MAIN_ADDRESS'] = np.where(retailer_site['RETAILER_SITE_ADDRESS2_address'].isna(), retailer_site['RETAILER_SITE_ADDRESS1_address'], np.where(retailer_site['RETAILER_SITE_ADDRESS1_address'].isna(), retailer_site['RETAILER_SITE_ADDRESS2_address'], None))

retailer_headquarters['MAIN_ADDRESS'] = np.where(retailer_headquarters['RETAILER_HEADQUARTER_address2_address'].isna(), retailer_headquarters['RETAILER_HEADQUARTER_address1_address'], np.where(retailer_headquarters['RETAILER_HEADQUARTER_address1_address'].isna(), retailer_headquarters['RETAILER_HEADQUARTER_address2_address'], None))


### GO_SALES_PRODUCT_FORECAST

In [55]:
sales_product_forcast['YEAR_MONTH'] = sales_product_forcast['YEAR_MONTH'].astype('string')

sales_product_forcast.dtypes

PRODUCT_NUMBER                                   int64
EXPECTED_VOLUME                                  int64
PRODUCT_number                                   int32
PRODUCT_INTRODUCTION_DATE_introduced            object
PRODUCT_PRODUCT_TYPE_code                       object
PRODUCT_PRODUCTION_COST_cost                    object
PRODUCT_MARGIN_margin                           object
PRODUCT_image_image                             object
PRODUCT_LANGUAGE_language                       object
PRODUCT_name_product                            object
PRODUCT_description_description                 object
TRIAL888_x                                      object
PRODUCT_PRODUCT_LINE_code                       object
PRODUCT_PRODUCT_TYPE_code_en                    object
TRIAL888_y                                      object
PRODUCT_PRODUCT_LINE_code_en                    object
TRIAL888                                        object
YEAR_MONTH                              string[python]
dtype: obj

## Opslaan historische data
Aan alle tabellen een kolom toevoegen met de datum van de wijziging en of dit de huidige versie is.

In [56]:
for df in all_dataframes:
    df['LAST_UPDATED'] = pd.to_datetime('today')
    df['CURRENT'] = 1

# Data opslaan in de database
De pandas dataframes opslaan in de mssql database

## year

In [57]:
print(f"Rows: {year.shape[0]}")
for index, row in year.iterrows():
    query = 'INSERT INTO Year (YEAR) VALUES (?)'
    values = index
    export_cursor.execute(query, values)

Rows: 301


## Unit

In [58]:
print(f"Rows: {unit.shape[0]}")

unit_code_sk = {}

for index, row in unit.iterrows():
    query = 'INSERT INTO Unit (UNIT_id, UNIT_COST_cost, UNIT_PRICE_price, UNIT_SALE_sale) VALUES (?,?,?,?);'
    values = (row['UNIT_id'], row['UNIT_COST_cost'], row['UNIT_PRICE_price'], row['UNIT_SALE_sale'])
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    unit_code_sk[row['UNIT_id']] = generated_primary_key


Rows: 43063


## date

In [59]:
print(f"Rows: {date.shape[0]}")
for index, row in date.iterrows():
    query = 'INSERT INTO Date (DATE_date) VALUES (?)'
    values = index
    export_cursor.execute(query, values)

Rows: 109574


## Sales_staff

In [60]:
print(f"Rows: {sales_staff.shape[0]}")

sales_staff_code_sk = {}

for index, row in sales_staff.iterrows():
    query = 'INSERT INTO Sales_staff (SALES_STAFF_code, SALES_STAFF_email, SALES_STAFF_extension, SALES_STAFF_POSITION_EN_position, SALES_STAFF_WORK_PHONE_work_phone, SALES_STAFF_DATE_HIRED_hired, SALES_STAFF_FAX, SALES_STAFF_FIRST_NAME_first_name, SALES_STAFF_LAST_NAME_last_name, SALES_STAFF_SALES_BRANCH_CODE_branch_code, SALES_STAFF_SALES_BRANCH_ADDRESS1_address, SALES_STAFF_SALES_BRANCH_ADDRESS2_address) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)'

    values = (row['SALES_STAFF_code'], row['SALES_STAFF_email'], row['SALES_STAFF_extension'], row['SALES_STAFF_POSITION_EN_position'], row['SALES_STAFF_WORK_PHONE_work_phone'], row['SALES_STAFF_DATE_HIRED_hired'], row['SALES_STAFF_FAX'], row['SALES_STAFF_FIRST_NAME_first_name'], row['SALES_STAFF_LAST_NAME_last_name'], row['SALES_STAFF_SALES_BRANCH_CODE_branch_code'], row['SALES_STAFF_SALES_BRANCH_ADDRESS1_address'], row['SALES_STAFF_SALES_BRANCH_ADDRESS2_address'])

    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    sales_staff_code_sk[row['SALES_STAFF_code']] = generated_primary_key

# foreign key manager SALES_STAFF_MANAGER_CODE_manager
for index, row in sales_staff.iterrows():
        if pd.notnull(row['SALES_STAFF_MANAGER_CODE_manager']):
            query = 'UPDATE Sales_staff SET SALES_STAFF_MANAGER_CODE_manager = ? WHERE SALES_STAFF_SK = ?'
            values = (sales_staff_code_sk[row['SALES_STAFF_MANAGER_CODE_manager']], sales_staff_code_sk[row['SALES_STAFF_code']])
            export_cursor.execute(query, values)

Rows: 102


## satisfaction_type

In [61]:
print(f"Rows: {satisfaction_type.shape[0]}")

satisfaction_type_code_sk = {}

for index, row in satisfaction_type.iterrows():
    query = 'INSERT INTO Satisfaction_type (SATISFACTION_TYPE_code, SATISFACTION_TYPE_DESCRIPTION, CURRENT_VALUE) VALUES (?,?,?)'
    values = (row['SATISFACTION_TYPE_code'], row['SATISFACTION_TYPE_DESCRIPTION'], 1)
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    satisfaction_type_code_sk[row['SATISFACTION_TYPE_code']] = generated_primary_key

Rows: 5


## Course

In [62]:
print(f"Rows: {course.shape[0]}")

course_code_sk = {}

for index, row in course.iterrows():
    query = 'INSERT INTO Course (COURSE_code, COURSE_DESCRIPTION, CURRENT_VALUE) VALUES (?,?,?)'
    values = (row['COURSE_code'], row['COURSE_DESCRIPTION'], 1)
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    course_code_sk[row['COURSE_code']] = generated_primary_key

Rows: 9


## order

In [63]:
print(f"Rows: {order.shape[0]}")

order_code_sk = {}

for index, row in order.iterrows():
    query = 'INSERT INTO "Order" (ORDER_order_number, ORDER_ORDER_METHOD_CODE_method_code, ORDER_ORDER_METHOD_EN_method) VALUES (?,?,?)'
    values = (row['ORDER_order_number'], row['ORDER_ORDER_METHOD_CODE_method_code'], row['ORDER_ORDER_METHOD_EN_method'])
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    order_code_sk[row['ORDER_order_number']] = generated_primary_key

Rows: 5360


## retailer_site

In [64]:
print(f"Rows: {retailer_site.shape[0]}")

# dictionary to store the primary key with the surrogate key. To be used for fk reference
retailer_site_code_sk_dict = {}

for index, row in retailer_site.iterrows():
    query = 'INSERT INTO Retailer_site (RETAILER_SITE_code, RETAILER_SITE_COUNTRY_CODE_country, RETAILER_SITE_CITY_city, RETAILER_SITE_REGION_region, RETAILER_SITE_POSTAL_ZONE_postal_zone, RETAILER_SITE_RETAILER_CODE_retailer_code, RETAILER_SITE_ACTIVE_INDICATOR_indicator, RETAILER_SITE_ADDRESS1_address, RETAILER_SITE_ADDRESS2_address) VALUES (?,?,?,?,?,?,?,?,?)'
    values = (row['RETAILER_SITE_code'], row['RETAILER_SITE_COUNTRY_CODE_country'], row['RETAILER_SITE_CITY_city'], row['RETAILER_SITE_REGION_region'], row['RETAILER_SITE_POSTAL_ZONE_postal_zone'], row['RETAILER_SITE_RETAILER_CODE_retailer_code'], row['RETAILER_SITE_ACTIVE_INDICATOR_indicator'], row['RETAILER_SITE_ADDRESS1_address'], row['RETAILER_SITE_ADDRESS2_address'])
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    retailer_site_code_sk_dict[row['RETAILER_SITE_code']] = generated_primary_key

Rows: 391


## sales_branch

In [65]:
print(f"Rows: {sales_branch.shape[0]}")

sales_branch_code_sk = {}

for index, row in sales_branch.iterrows():
    query = 'INSERT INTO Sales_branch (SALES_BRANCH_code, SALES_BRANCH_COUNTRY_CODE_country, SALES_BRANCH_REGION_region, SALES_BRANCH_CITY_city, SALES_BRANCH_POSTAL_ZONE_postal_zone, SALES_BRANCH_ADDRESS1_address, SALES_BRANCH_ADDRESS2_address) VALUES (?,?,?,?,?,?,?)'
    values = (row['SALES_BRANCH_code'], row['SALES_BRANCH_COUNTRY_CODE_country'], row['SALES_BRANCH_REGION_region'], row['SALES_BRANCH_CITY_city'], row['SALES_BRANCH_POSTAL_ZONE_postal_zone'], row['SALES_BRANCH_ADDRESS1_address'], row['SALES_BRANCH_ADDRESS2_address'])
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    sales_branch_code_sk[row['SALES_BRANCH_code']] = generated_primary_key

Rows: 28


## retailer_contact

In [66]:
print(f"Rows: {retailer_contact.shape[0]}")

retailer_contact_code_sk = {}

for index, row in retailer_contact.iterrows():
    query = 'INSERT INTO Retailer_contact (RETAILER_CONTACT_code, RETAILER_CONTACT_email, RETAILER_CONTACT_RETAILER_SITE_CODE_site_code, RETAILER_CONTACT_JOB_POSITION_EN_position, RETAILER_CONTACT_EXTENSION_extension, RETAILER_CONTACT_FAX_fax, RETAILER_CONTACT_FIRST_NAME_first_name, RETAILER_CONTACT_LAST_NAME_last_name) VALUES (?,?,?,?,?,?,?,?)'
    
    values = (row['RETAILER_CONTACT_code'], row['RETAILER_CONTACT_email'], retailer_site_code_sk_dict[row['RETAILER_CONTACT_RETAILER_SITE_CODE_site_code']], row['RETAILER_CONTACT_JOB_POSITION_EN_position'], row['RETAILER_CONTACT_EXTENSION_extension'], row['RETAILER_CONTACT_FAX_fax'], row['RETAILER_CONTACT_FIRST_NAME_first_name'], row['RETAILER_CONTACT_LAST_NAME_last_name'])
    
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())    
    retailer_contact_code_sk[row['RETAILER_CONTACT_code']] = generated_primary_key

Rows: 391


## retailer

In [67]:
print(f"Rows: {retailer.shape[0]}")

retailer_code_sk = {}

for index, row in retailer.iterrows():
    query = 'INSERT INTO Retailer (RETAILER_code, RETAILER_name, RETAILER_COMPANY_CODE_MR_company, RETAILER_RETAILER_TYPE_code, RETAILER_RETAILER_TYPE_EN, CURRENT_VALUE) VALUES (?,?,?,?,?,?)'
    values = (row['RETAILER_code'], row['RETAILER_name'], row['RETAILER_COMPANY_CODE_MR_company'], row['RETAILER_RETAILER_TYPE_code'], row['RETAILER_RETAILER_TYPE_EN'], 1)
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    retailer_code_sk[row['RETAILER_code']] = generated_primary_key

Rows: 109


## product

In [68]:
print(f"Rows: {product.shape[0]}")

product_code_sk = {}

for index, row in product.iterrows():
    query = 'INSERT INTO Product (PRODUCT_number, PRODUCT_name_product, PRODUCT_description_description, PRODUCT_image_image, PRODUCT_INTRODUCTION_DATE_introduced, PRODUCT_PRODUCTION_COST_cost, PRODUCT_MARGIN_margin, PRODUCT_LANGUAGE_language, PRODUCT_PRODUCT_LINE_code, PRODUCT_PRODUCT_LINE_code_en, PRODUCT_PRODUCT_TYPE_code, PRODUCT_PRODUCT_TYPE_code_en) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)'
    values = (row['PRODUCT_number'], row['PRODUCT_name_product'], row['PRODUCT_description_description'], row['PRODUCT_image_image'], row['PRODUCT_INTRODUCTION_DATE_introduced'], row['PRODUCT_PRODUCTION_COST_cost'], row['PRODUCT_MARGIN_margin'], row['PRODUCT_LANGUAGE_language'], row['PRODUCT_PRODUCT_LINE_code'], row['PRODUCT_PRODUCT_LINE_code_en'], row['PRODUCT_PRODUCT_TYPE_code'], row['PRODUCT_PRODUCT_TYPE_code_en'])
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    product_code_sk[row['PRODUCT_number']] = generated_primary_key

Rows: 115


## order_details

In [69]:
print(f"Rows: {order_details.shape[0]}")

order_detail_code_sk = {}

for index, row in order_details.iterrows():
    query = 'INSERT INTO Order_details (ORDER_DETAILS_code, ORDER_DETAILS_QUANTITY_quantity, ORDER_DETAILS_TOTAL_COST_total, ORDER_DETAILS_TOTAL_MARGIN_margin,  ORDER_DETAILS_ORDER_NUMBER_order, ORDER_DETAILS_PRODUCT_NUMBER_product, ORDER_DETAILS_UNIT_ID_unit) VALUES (?,?,?,?,?,?,?)'
    values = (row['ORDER_DETAIL_code'], row['ORDER_DETAILS_QUANTITY_quantity'], row['ORDER_DETAILS_TOTAL_COST_total'], row['ORDER_DETAILS_TOTAL_MARGIN_margin'], order_code_sk[row['ORDER_DETAILS_ORDER_NUMBER_order']], product_code_sk[row['ORDER_DETAILS_PRODUCT_NUMBER_product']], row['UNIT_id'])
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    order_detail_code_sk[row['ORDER_DETAIL_code']] = generated_primary_key
    

Rows: 43063


## returned_item

In [70]:
print(f"Rows: {returned_item.shape[0]}")

for index, row in returned_item.iterrows():
    query = 'INSERT INTO Returned_item (RETURNED_ITEM_code, RETURNED_ITEM_DATE, RETURNED_ITEM_QUANTITY, RETURNED_ITEM_ORDER_DETAIL_CODE, RETURNED_ITEM_RETURN_REASON_code, RETURNED_ITEM_RETURN_REASON_description_en, RETURNED_ITEM_RETURNED_ITEMS_TOTAL_PRICE) VALUES (?,?,?,?,?,?,?)'
    values = (row['RETURNED_ITEM_code'], row['RETURNED_ITEM_DATE'], row['RETURNED_ITEM_QUANTITY'], order_detail_code_sk[row['RETURNED_ITEM_ORDER_DETAIL_CODE']], row['RETURNED_ITEM_RETURN_REASON_code'], row['RETURNED_ITEM_RETURN_REASON_description_en'], row['RETURNED_ITEM_RETURNED_ITEMS_TOTAL_PRICE'])
    export_cursor.execute(query, values)

Rows: 706


## sales_target_data

In [71]:
print(f"Rows: {sales_target_data.shape[0]}")
for index, row in sales_target_data.iterrows():
    query = 'INSERT INTO SALES_TARGETDATA (SALES_TARGETDATA_SALES_YEAR, SALES_TARGETDATA_SALES_PERIOD, SALES_TARGETDATA_RETAILER_NAME, SALES_TARGETDATA_SALES_TARGET, SALES_TARGETDATA_TARGET_COST, SALES_TARGETDATA_TARGET_MARGIN, SALES_TARGETDATA_SALES_STAFF_CODE, SALES_TARGETDATA_PRODUCT_NUMBER, SALES_TARGETDATA_RETAILER_CODE, SALES_TARGETDATA_SALES_TARGETDATA_ID) VALUES (?,?,?,?,?,?,?,?,?, ?)'
    values = (row['SALES_TARGETDATA_SALES_YEAR'], row['SALES_TARGETDATA_SALES_PERIOD'], row['SALES_TARGETDATA_RETAILER_NAME'], row['SALES_TARGETDATA_SALES_TARGET'], row['SALES_TARGET_DATA_TARGET_COST'], row['SALES_TARGET_DATA_TARGET_MARGIN'],  sales_staff_code_sk[row['SALES_TARGETDATA_SALES_STAFF_CODE']], product_code_sk[row['PRODUCT_number']], retailer_code_sk[row['RETAILER_code']], row['Id'])
    export_cursor.execute(query, values)

Rows: 39530


## training

In [72]:
print(f"Rows: {training.shape[0]}")
for index, row in training.iterrows():
    query = 'INSERT INTO Training (TRAINING_SALES_STAFF_CODE, TRAINING_COURSE_CODE, TRAINING_YEAR) VALUES (?,?,?)'
    values = (sales_staff_code_sk[row['TRAINING_SALES_STAFF_CODE']], course_code_sk[row['TRAINING_COURSE_CODE']], row['TRAINING_YEAR'])
    export_cursor.execute(query, values)

Rows: 402


## satisfaction

In [73]:
print(f"Rows: {satisfaction.shape[0]}")
for index, row in satisfaction.iterrows():
    query = 'INSERT INTO Satisfaction (SATISFACTION_SALES_STAFF_CODE, SATISFACTION_SATISFACTION_TYPE_CODE, SATISFACTION_YEAR) VALUES (?,?,?)'
    values = (sales_staff_code_sk[row['SATISFACTION_SALES_STAFF_CODE']], satisfaction_type_code_sk[row['SATISFACTION_SATISFACTION_TYPE_CODE']], row['SATISFACTION_YEAR'])
    export_cursor.execute(query, values)

Rows: 301


## order_header

In [74]:
print(f"Rows: {order_header.shape[0]}")

for index, row in order_header.iterrows():
    query = 'INSERT INTO Order_header (ORDER_HEADER_number, ORDER_HEADER_SALES_STAFF_CODE, ORDER_HEADER_SALES_BRANCH_CODE, ORDER_HEADER_RETAILER_SITE_CODE, ORDER_HEADER_RETAILER_CONTACT_CODE, ORDER_HEADER_RETAILER_CODE, ORDER_HEADER_ORDER_DATE, ORDER_HEADER_ORDER_order_number) VALUES (?,?,?,?,?,?,?, ?)'
    values = (row['ORDER_HEADER_number'], sales_staff_code_sk[row['ORDER_HEADER_SALES_STAFF_CODE']], sales_branch_code_sk[row['ORDER_HEADER_SALES_BRANCH_CODE']], retailer_site_code_sk_dict[row['ORDER_HEADER_RETAILER_SITE_CODE']], retailer_contact_code_sk[row['ORDER_HEADER_RETAILER_CONTACT_CODE']], retailer_code_sk[row['RETAILER_CODE']], row['ORDER_HEADER_ORDER_DATE'], order_code_sk[row['ORDER_HEADER_number']])
    export_cursor.execute(query, values)

Rows: 4965


## retailer_segment

In [75]:
print(f"Rows: {retailer_segment.shape[0]}")
for index, row in retailer_segment.iterrows():
    query = 'INSERT INTO Retailer_segment (RETAILER_SEGMENT_segment_code, RETAILER_SEGMENT_language, RETAILER_SEGMENT_segment_name, RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description, SEGMENT_DESCRIPTION_description_short, CURRENT_VALUE) VALUES (?,?,?,?,?,?)'
    values = (row['RETAILER_SEGMENT_segment_code'], row['RETAILER_SEGMENT_language'], row['RETAILER_SEGMENT_segment_name'], row['RETAILER_SEGMENT_SEGMENT_DESCRIPTION_description'], row['SEGMENT_DESCRIPTION_SHORT'], 1)
    export_cursor.execute(query, values)

Rows: 12


## retailer_headquarters

In [76]:
print(f"Rows: {retailer_headquarters.shape[0]}")

retailer_headquarters_code_sk = {}

for index, row in retailer_headquarters.iterrows():
    query = 'INSERT INTO Retailer_headquarter (RETAILER_HEADQUARTER_codemr, RETAIL_HEADQUARTER_retailer_name, RETAILER_HEADQUARTER_address1_address, RETAILER_HEADQUARTER_address2_address, RETAILER_HEADQUARTER_city_city, RETAILER_HEADQUARTER_region_region, RETAILER_HEADQUARTER_postal_zone_postal_zone, RETAILER_HEADQUARTER_country_code_country, RETAILER_HEADQUARTER_phone_phone, RETAILER_HEADQUARTER_fax_fax, RETAILER_HEADQUARTER_segment_code, RETAILER_HEADQUARTER_main_address_address) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)'
    values = (row['RETAILER_HEADQUARTER_codemr'], row['RETAIL_HEADQUARTER_retailer_name'], row['RETAILER_HEADQUARTER_address1_address'], row['RETAILER_HEADQUARTER_address2_address'], row['RETAILER_HEADQUARTER_city_city'], row['RETAILER_HEADQUARTER_region_region'], row['RETAILER_HEADQUARTER_postal_zone_postal_zone'], row['RETAILER_HEADQUARTER_country_code_country'], row['RETAILER_HEADQUARTER_phone_phone'], row['RETAILER_HEADQUARTER_fax_fax'], row['RETAILER_HEADQUARTER_segment_code'], row['MAIN_ADDRESS'])
    export_cursor.execute(query, values)
    
    # Execute the SELECT statement to retrieve the generated primary key
    query = 'SELECT @@IDENTITY AS GENERATED_KEY'
    export_cursor.execute(query)
    generated_primary_key = int(export_cursor.fetchval())
    retailer_headquarters_code_sk[row['RETAILER_HEADQUARTER_codemr']] = generated_primary_key

Rows: 414


## Go_sales_inventory_levels

In [77]:
print(f"Rows: {go_sales_inventory_levels.shape[0]}")
for index, row in go_sales_inventory_levels.iterrows():
    query = 'INSERT INTO GO_SALES_INVENTORY_LEVELS (GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER, GO_SALES_INVENTORY_LEVELS_YEAR_MONTH, GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT, GO_SALES_INVENTORY_LEVELS_id) VALUES (?,?,?, ?)'
    values = (product_code_sk[row['GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER']], row['GO_SALES_INVENTORY_LEVELS_YEAR_MONTH'], row['GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT'], index)
    export_cursor.execute(query, values)

Rows: 3888


## Go_sales_product_forecast

In [78]:
print(f"Rows: {sales_product_forecast.shape[0]}")

# converting expected cost and expected margin to float
sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_COST'] = sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_COST'].astype('float')
sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN'] = sales_product_forecast['GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN'].astype('float')

for index, row in sales_product_forecast.iterrows():
    query = 'INSERT INTO GO_SALES_PRODUCT_FORECAST (GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER, GO_SALES_PRODUCT_FORECAST_YEAR_MONTH, GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME, GO_SALES_PRODUCT_FORECAST_EXPECTED_COST, GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN, GO_SALES_PRODUCT_FORECAST_id) VALUES (?,?,?,?,?,?)'
    values = (product_code_sk[row['GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER']], row['GO_SALES_PRODUCT_FORECAST_YEAR_MONTH'],row['GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME'], row['GO_SALES_PRODUCT_FORECAST_EXPECTED_COST'], row['GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN'], index)
    export_cursor.execute(query, values)

Rows: 3872


## Sales_demographic

In [80]:
print(f"Rows: {sales_demographic.shape[0]}")

for index, row in sales_demographic.iterrows():
    query = 'INSERT INTO SALES_DEMOGRAPHIC (SALES_DEMOGRAPHIC_AGE_GROUP_CODE, SALES_DEMOGRAPHIC_UPPER_AGE, SALES_DEMOGRAPHIC_LOWER_AGE, SALES_DEMOGRAPHIC_code,SALES_DEMOGRAPHIC_RETAILER_CODE,SALES_DEMOGRAPHIC_SALES_PERCENT) VALUES (?,?,?,?,?,?)'
    values = (row['SALES_DEMOGRAPHIC_AGE_GROUP_CODE'], row['SALES_DEMOGRAPHIC_UPPER_AGE'], row['SALES_DEMOGRAPHIC_LOWER_AGE'], row['SALES_DEMOGRAPHIC_code'], retailer_headquarters_code_sk[row['SALES_DEMOGRAPHIC_RETAILER_CODE']], row['SALES_DEMOGRAPHIC_SALES_PERCENT'])
    export_cursor.execute(query, values)   

Rows: 2484


KeyError: 2

## clean up

In [ ]:
# close cursor
export_cursor.close()

# close connection
export_conn.close()